# 2. CREATE & RUN CLASTER

CLASTER is, at its core, a deep convolutional neural network aimed to translate a given chromatin landscape and its matching 3D structure to the corresponding nascent RNA landscape. 


The network consists of:

- **Feature extractors**: two separate feature extractors with a number of convolutional layers implementing dilated convolutions and some residual connections. 
    - The first one extracts patterns from the chromatin landscape, described by combinations of ATAC-seq, H3K4me3, H3K27ac and H3K27me3 enrichments. 
    - The second one extracts features from the Micro-C maps (e.g. point-like contacts, compartments or domains).
- **Fusion module**: The extracted high-level, abstract features are then combined in a number of dense layers.
- **Output module**: A final set of dense layers maps the feature vectors to the targets in a regression task per node, which will represent the EU-seq signal at a distance from the TSS of the gene defining the sample (located in the middle).

CLASTER was built using the EIR framework, which makes it easy to replicate and adapt to new tasks, so feel free to do so! Documentation on EIR can be found at https://eir.readthedocs.io/en/latest/. Have a look at the tutorials to get a feel for the config files required and all possibilities that EIR offers. The framework uses hydra (https://hydra.cc/docs/intro/) to manage a set of configuration files, which allow you to streamline the process.

First, install eir in your environment using pip

> *Note: If you created claster_env, it would be nice to proceed in that environment.*

``` bash
! pip install eir-dl 
```

In [ ]:
from pathlib import Path
import pandas as pd

### Create config files:

The config files used to build CLASTER can be created as follows. This code creates all config files at once so that we can compare them and replicate the results.
We will mainly need:
- Global config: setting global parameters like the device, learning rate, batch size, whether we want to compute feature importance scores using attributions...
- Input configs: as many as feature extractors/ data modalities we have or want to integrate. Describes the feature extractor architecture
- Fusion config: describing how to we want to merge the features extracted from the different branches.
- Output config: describing the type of output we expect, our target file, the loss we want to optimize against and the names of the nodes.

For a more detailed description, please go through EIR's documentation.

In [ ]:
config_paths = [Path("../configurations/conf_pure_conv/"), 
                Path("../configurations/conf_pure_conv_predict/"),
                Path("../configurations/conf_pure_conv_predict_perturbations/"),
                Path("../configurations/conf_microc_pure_conv/"),
                Path("../configurations/conf_microc_rotated_pure_conv/"),
                Path("../configurations/conf_microc_pure_conv_predict/"),
                Path("../configurations/conf_microc_rotated_pure_conv_predict/"),
                Path("../configurations/conf_only_chrom_attention/"),
                Path("../configurations/conf_only_chrom_attention_predict/"),
                Path("../configurations/conf_microc_pure_conv_latents/"),
                Path("../configurations/conf_pure_conv_predict_perturbations_H3K27ac/")]

for config_path in config_paths:
  config_path.mkdir(parents=True, exist_ok=True)

training_yaml_contents = {"globals.yaml":"""
output_folder: ./runs/gene_expression_only_chrom_pure_conv/  #gene_expression_prediction_no_H3K27ac_uncoupled/     #gene_expression_cnn_1kbp_401_bins_2_cond_reloaded/ 
manual_valid_ids_file: ./annotations/manual_validation_ids_chr17.txt  #manual_validation_ids_chr17_uncoupled.txt
checkpoint_interval: 30300 # 100 epochs #60000
sample_interval: 30300 #60000
n_epochs: 120
batch_size: 64
optimizer: "adamw"
lr: 0.0001 #0.0001
device: "cuda"
compute_attributions: true
# latent_sampling: 
#   layers_to_sample:
#     - "input_modules.contact_maps.feature_extractor.conv.0.conv_1"
#     - "input_modules.contact_maps.feature_extractor.conv.1.conv_2"
#     - "input_modules.contact_maps.feature_extractor.conv.2.conv_2"
#     - "input_modules.contact_maps.feature_extractor.conv.3.conv_2"
# attribution_background_samples: 512
# attributions_every_sample_factor: 1
#pretrained_checkpoint: best_models/gene_expression_exformer_unlimited_chrom_and_micro_with_attention_model_117600_perf-average=0.8435.pt 

""",
"input_cnn.yaml": """ 
input_info:
  input_source: ./inputs/landscape_arrays/training/ 
  input_name: gene_expression
  input_type: array

model_config:
  model_type: cnn
  #pre_normalization: "instancenorm"
  model_init_config:
    num_output_features: 512 # before fc_repr_dim
    layers: [4,4]
    kernel_height: 1
    down_stride_width: 2
    first_stride_expansion_width: 1
    first_kernel_expansion_height: 4 #5
    kernel_width: 10  #10
    dilation_factor_width: 2
    dilation_factor_height: 1
    channel_exp_base: 5  #3 #-1
    first_channel_expansion: 2
    rb_do: .3
    stochastic_depth_p: .1
    attention_inclusion_cutoff: 1 #50 #256

""",
"fusion.yaml": """
model_config:
  fc_do: 0.4
  fc_task_dim: 256
  layers:
  - 2
  rb_do: 0.4
  stochastic_depth_p: 0.5
model_type: "mlp-residual"
""",
"outputs_2_cond.yaml":"""
output_info:
  output_name: expression_output
  output_source: ./targets/training_targets.csv 
  output_type: tabular

model_config: # <- new
  model_type: linear # <- new

output_type_info:
  con_loss_name: "SmoothL1Loss"
  target_con_columns:
  - "-200_ctrl"
  - "-199_ctrl"
  - "-198_ctrl"
  - "-197_ctrl"
  - "-196_ctrl"
  - "-195_ctrl"
  - "-194_ctrl"
  - "-193_ctrl"
  - "-192_ctrl"
  - "-191_ctrl"
  - "-190_ctrl"
  - "-189_ctrl"
  - "-188_ctrl"
  - "-187_ctrl"
  - "-186_ctrl"
  - "-185_ctrl"
  - "-184_ctrl"
  - "-183_ctrl"
  - "-182_ctrl"
  - "-181_ctrl"
  - "-180_ctrl"
  - "-179_ctrl"
  - "-178_ctrl"
  - "-177_ctrl"
  - "-176_ctrl"
  - "-175_ctrl"
  - "-174_ctrl"
  - "-173_ctrl"
  - "-172_ctrl"
  - "-171_ctrl"
  - "-170_ctrl"
  - "-169_ctrl"
  - "-168_ctrl"
  - "-167_ctrl"
  - "-166_ctrl"
  - "-165_ctrl"
  - "-164_ctrl"
  - "-163_ctrl"
  - "-162_ctrl"
  - "-161_ctrl"
  - "-160_ctrl"
  - "-159_ctrl"
  - "-158_ctrl"
  - "-157_ctrl"
  - "-156_ctrl"
  - "-155_ctrl"
  - "-154_ctrl"
  - "-153_ctrl"
  - "-152_ctrl"
  - "-151_ctrl"
  - "-150_ctrl"
  - "-149_ctrl"
  - "-148_ctrl"
  - "-147_ctrl"
  - "-146_ctrl"
  - "-145_ctrl"
  - "-144_ctrl"
  - "-143_ctrl"
  - "-142_ctrl"
  - "-141_ctrl"
  - "-140_ctrl"
  - "-139_ctrl"
  - "-138_ctrl"
  - "-137_ctrl"
  - "-136_ctrl"
  - "-135_ctrl"
  - "-134_ctrl"
  - "-133_ctrl"
  - "-132_ctrl"
  - "-131_ctrl"
  - "-130_ctrl"
  - "-129_ctrl"
  - "-128_ctrl"
  - "-127_ctrl"
  - "-126_ctrl"
  - "-125_ctrl"
  - "-124_ctrl"
  - "-123_ctrl"
  - "-122_ctrl"
  - "-121_ctrl"
  - "-120_ctrl"
  - "-119_ctrl"
  - "-118_ctrl"
  - "-117_ctrl"
  - "-116_ctrl"
  - "-115_ctrl"
  - "-114_ctrl"
  - "-113_ctrl"
  - "-112_ctrl"
  - "-111_ctrl"
  - "-110_ctrl"
  - "-109_ctrl"
  - "-108_ctrl"
  - "-107_ctrl"
  - "-106_ctrl"
  - "-105_ctrl"
  - "-104_ctrl"
  - "-103_ctrl"
  - "-102_ctrl"
  - "-101_ctrl"
  - "-100_ctrl"
  - "-99_ctrl"
  - "-98_ctrl"
  - "-97_ctrl"
  - "-96_ctrl"
  - "-95_ctrl"
  - "-94_ctrl"
  - "-93_ctrl"
  - "-92_ctrl"
  - "-91_ctrl"
  - "-90_ctrl"
  - "-89_ctrl"
  - "-88_ctrl"
  - "-87_ctrl"
  - "-86_ctrl"
  - "-85_ctrl"
  - "-84_ctrl"
  - "-83_ctrl"
  - "-82_ctrl"
  - "-81_ctrl"
  - "-80_ctrl"
  - "-79_ctrl"
  - "-78_ctrl"
  - "-77_ctrl"
  - "-76_ctrl"
  - "-75_ctrl"
  - "-74_ctrl"
  - "-73_ctrl"
  - "-72_ctrl"
  - "-71_ctrl"
  - "-70_ctrl"
  - "-69_ctrl"
  - "-68_ctrl"
  - "-67_ctrl"
  - "-66_ctrl"
  - "-65_ctrl"
  - "-64_ctrl"
  - "-63_ctrl"
  - "-62_ctrl"
  - "-61_ctrl"
  - "-60_ctrl"
  - "-59_ctrl"
  - "-58_ctrl"
  - "-57_ctrl"
  - "-56_ctrl"
  - "-55_ctrl"
  - "-54_ctrl"
  - "-53_ctrl"
  - "-52_ctrl"
  - "-51_ctrl"
  - "-50_ctrl"
  - "-49_ctrl"
  - "-48_ctrl"
  - "-47_ctrl"
  - "-46_ctrl"
  - "-45_ctrl"
  - "-44_ctrl"
  - "-43_ctrl"
  - "-42_ctrl"
  - "-41_ctrl"
  - "-40_ctrl"
  - "-39_ctrl"
  - "-38_ctrl"
  - "-37_ctrl"
  - "-36_ctrl"
  - "-35_ctrl"
  - "-34_ctrl"
  - "-33_ctrl"
  - "-32_ctrl"
  - "-31_ctrl"
  - "-30_ctrl"
  - "-29_ctrl"
  - "-28_ctrl"
  - "-27_ctrl"
  - "-26_ctrl"
  - "-25_ctrl"
  - "-24_ctrl"
  - "-23_ctrl"
  - "-22_ctrl"
  - "-21_ctrl"
  - "-20_ctrl"
  - "-19_ctrl"
  - "-18_ctrl"
  - "-17_ctrl"
  - "-16_ctrl"
  - "-15_ctrl"
  - "-14_ctrl"
  - "-13_ctrl"
  - "-12_ctrl"
  - "-11_ctrl"
  - "-10_ctrl"
  - "-9_ctrl"
  - "-8_ctrl"
  - "-7_ctrl"
  - "-6_ctrl"
  - "-5_ctrl"
  - "-4_ctrl"
  - "-3_ctrl"
  - "-2_ctrl"
  - "-1_ctrl"
  - "0_ctrl"
  - "1_ctrl"
  - "2_ctrl"
  - "3_ctrl"
  - "4_ctrl"
  - "5_ctrl"
  - "6_ctrl"
  - "7_ctrl"
  - "8_ctrl"
  - "9_ctrl"
  - "10_ctrl"
  - "11_ctrl"
  - "12_ctrl"
  - "13_ctrl"
  - "14_ctrl"
  - "15_ctrl"
  - "16_ctrl"
  - "17_ctrl"
  - "18_ctrl"
  - "19_ctrl"
  - "20_ctrl"
  - "21_ctrl"
  - "22_ctrl"
  - "23_ctrl"
  - "24_ctrl"
  - "25_ctrl"
  - "26_ctrl"
  - "27_ctrl"
  - "28_ctrl"
  - "29_ctrl"
  - "30_ctrl"
  - "31_ctrl"
  - "32_ctrl"
  - "33_ctrl"
  - "34_ctrl"
  - "35_ctrl"
  - "36_ctrl"
  - "37_ctrl"
  - "38_ctrl"
  - "39_ctrl"
  - "40_ctrl"
  - "41_ctrl"
  - "42_ctrl"
  - "43_ctrl"
  - "44_ctrl"
  - "45_ctrl"
  - "46_ctrl"
  - "47_ctrl"
  - "48_ctrl"
  - "49_ctrl"
  - "50_ctrl"
  - "51_ctrl"
  - "52_ctrl"
  - "53_ctrl"
  - "54_ctrl"
  - "55_ctrl"
  - "56_ctrl"
  - "57_ctrl"
  - "58_ctrl"
  - "59_ctrl"
  - "60_ctrl"
  - "61_ctrl"
  - "62_ctrl"
  - "63_ctrl"
  - "64_ctrl"
  - "65_ctrl"
  - "66_ctrl"
  - "67_ctrl"
  - "68_ctrl"
  - "69_ctrl"
  - "70_ctrl"
  - "71_ctrl"
  - "72_ctrl"
  - "73_ctrl"
  - "74_ctrl"
  - "75_ctrl"
  - "76_ctrl"
  - "77_ctrl"
  - "78_ctrl"
  - "79_ctrl"
  - "80_ctrl"
  - "81_ctrl"
  - "82_ctrl"
  - "83_ctrl"
  - "84_ctrl"
  - "85_ctrl"
  - "86_ctrl"
  - "87_ctrl"
  - "88_ctrl"
  - "89_ctrl"
  - "90_ctrl"
  - "91_ctrl"
  - "92_ctrl"
  - "93_ctrl"
  - "94_ctrl"
  - "95_ctrl"
  - "96_ctrl"
  - "97_ctrl"
  - "98_ctrl"
  - "99_ctrl"
  - "100_ctrl"
  - "101_ctrl"
  - "102_ctrl"
  - "103_ctrl"
  - "104_ctrl"
  - "105_ctrl"
  - "106_ctrl"
  - "107_ctrl"
  - "108_ctrl"
  - "109_ctrl"
  - "110_ctrl"
  - "111_ctrl"
  - "112_ctrl"
  - "113_ctrl"
  - "114_ctrl"
  - "115_ctrl"
  - "116_ctrl"
  - "117_ctrl"
  - "118_ctrl"
  - "119_ctrl"
  - "120_ctrl"
  - "121_ctrl"
  - "122_ctrl"
  - "123_ctrl"
  - "124_ctrl"
  - "125_ctrl"
  - "126_ctrl"
  - "127_ctrl"
  - "128_ctrl"
  - "129_ctrl"
  - "130_ctrl"
  - "131_ctrl"
  - "132_ctrl"
  - "133_ctrl"
  - "134_ctrl"
  - "135_ctrl"
  - "136_ctrl"
  - "137_ctrl"
  - "138_ctrl"
  - "139_ctrl"
  - "140_ctrl"
  - "141_ctrl"
  - "142_ctrl"
  - "143_ctrl"
  - "144_ctrl"
  - "145_ctrl"
  - "146_ctrl"
  - "147_ctrl"
  - "148_ctrl"
  - "149_ctrl"
  - "150_ctrl"
  - "151_ctrl"
  - "152_ctrl"
  - "153_ctrl"
  - "154_ctrl"
  - "155_ctrl"
  - "156_ctrl"
  - "157_ctrl"
  - "158_ctrl"
  - "159_ctrl"
  - "160_ctrl"
  - "161_ctrl"
  - "162_ctrl"
  - "163_ctrl"
  - "164_ctrl"
  - "165_ctrl"
  - "166_ctrl"
  - "167_ctrl"
  - "168_ctrl"
  - "169_ctrl"
  - "170_ctrl"
  - "171_ctrl"
  - "172_ctrl"
  - "173_ctrl"
  - "174_ctrl"
  - "175_ctrl"
  - "176_ctrl"
  - "177_ctrl"
  - "178_ctrl"
  - "179_ctrl"
  - "180_ctrl"
  - "181_ctrl"
  - "182_ctrl"
  - "183_ctrl"
  - "184_ctrl"
  - "185_ctrl"
  - "186_ctrl"
  - "187_ctrl"
  - "188_ctrl"
  - "189_ctrl"
  - "190_ctrl"
  - "191_ctrl"
  - "192_ctrl"
  - "193_ctrl"
  - "194_ctrl"
  - "195_ctrl"
  - "196_ctrl"
  - "197_ctrl"
  - "198_ctrl"
  - "199_ctrl"
  - "200_ctrl"
  """
}

test_yaml_contents = {"fusion.yaml":"""  
model_config:
  fc_do: 0.4
  fc_task_dim: 256
  layers:
  - 2
  rb_do: 0.4
  stochastic_depth_p: 0.5
model_type: "mlp-residual"
""",
"globals.yaml": """

checkpoint_interval: 30300 # 100 epochs #60000
sample_interval: 30300 #60000
n_epochs: 120
batch_size: 64
optimizer: "adamw"
lr: 0.0001 #0.0001
device: "cuda"
compute_attributions: true

""",
"input_cnn.yaml": """
input_info:
  input_source: ./inputs/landscape_arrays/test/  #inserted_enhancer_test #inputs/silenced_arrays/silenced_arrays_H2B_S.D 
  #./data/parsed_data/inputs/arrays_train_100bp_no_H3K27ac/ #arrays_train_100bp_no_H3K27ac_uncoupled/
  input_name: gene_expression
  input_type: array

model_config:
  model_type: cnn
  #pre_normalization: "instancenorm"
  model_init_config:
    num_output_features: 512 # before fc_repr_dim
    layers: [4,4]
    kernel_height: 1
    down_stride_width: 2
    first_stride_expansion_width: 1
    first_kernel_expansion_height: 4 #5
    kernel_width: 10  #10
    dilation_factor_width: 2
    dilation_factor_height: 1
    channel_exp_base: 5  #3 #-1
    first_channel_expansion: 2
    rb_do: .3
    stochastic_depth_p: .1
    attention_inclusion_cutoff: 1 #50 #256
    """,
"outputs_2_cond.yaml": """
output_info:
  output_name: expression_output
  output_source: ./targets/test_targets.csv 
  #./data/parsed_data/targets/target_arrays_perturbational_inserted_enhancer.csv #target_arrays_perturbational_S.D.csv #target_arrays_1kbp_401_bins_2_conditions_decareads_abs.csv
  output_type: tabular

model_config: # <- new
  model_type: linear # <- new

output_type_info:
  con_loss_name: "SmoothL1Loss"
  target_con_columns:
  - "-200_ctrl"
  - "-199_ctrl"
  - "-198_ctrl"
  - "-197_ctrl"
  - "-196_ctrl"
  - "-195_ctrl"
  - "-194_ctrl"
  - "-193_ctrl"
  - "-192_ctrl"
  - "-191_ctrl"
  - "-190_ctrl"
  - "-189_ctrl"
  - "-188_ctrl"
  - "-187_ctrl"
  - "-186_ctrl"
  - "-185_ctrl"
  - "-184_ctrl"
  - "-183_ctrl"
  - "-182_ctrl"
  - "-181_ctrl"
  - "-180_ctrl"
  - "-179_ctrl"
  - "-178_ctrl"
  - "-177_ctrl"
  - "-176_ctrl"
  - "-175_ctrl"
  - "-174_ctrl"
  - "-173_ctrl"
  - "-172_ctrl"
  - "-171_ctrl"
  - "-170_ctrl"
  - "-169_ctrl"
  - "-168_ctrl"
  - "-167_ctrl"
  - "-166_ctrl"
  - "-165_ctrl"
  - "-164_ctrl"
  - "-163_ctrl"
  - "-162_ctrl"
  - "-161_ctrl"
  - "-160_ctrl"
  - "-159_ctrl"
  - "-158_ctrl"
  - "-157_ctrl"
  - "-156_ctrl"
  - "-155_ctrl"
  - "-154_ctrl"
  - "-153_ctrl"
  - "-152_ctrl"
  - "-151_ctrl"
  - "-150_ctrl"
  - "-149_ctrl"
  - "-148_ctrl"
  - "-147_ctrl"
  - "-146_ctrl"
  - "-145_ctrl"
  - "-144_ctrl"
  - "-143_ctrl"
  - "-142_ctrl"
  - "-141_ctrl"
  - "-140_ctrl"
  - "-139_ctrl"
  - "-138_ctrl"
  - "-137_ctrl"
  - "-136_ctrl"
  - "-135_ctrl"
  - "-134_ctrl"
  - "-133_ctrl"
  - "-132_ctrl"
  - "-131_ctrl"
  - "-130_ctrl"
  - "-129_ctrl"
  - "-128_ctrl"
  - "-127_ctrl"
  - "-126_ctrl"
  - "-125_ctrl"
  - "-124_ctrl"
  - "-123_ctrl"
  - "-122_ctrl"
  - "-121_ctrl"
  - "-120_ctrl"
  - "-119_ctrl"
  - "-118_ctrl"
  - "-117_ctrl"
  - "-116_ctrl"
  - "-115_ctrl"
  - "-114_ctrl"
  - "-113_ctrl"
  - "-112_ctrl"
  - "-111_ctrl"
  - "-110_ctrl"
  - "-109_ctrl"
  - "-108_ctrl"
  - "-107_ctrl"
  - "-106_ctrl"
  - "-105_ctrl"
  - "-104_ctrl"
  - "-103_ctrl"
  - "-102_ctrl"
  - "-101_ctrl"
  - "-100_ctrl"
  - "-99_ctrl"
  - "-98_ctrl"
  - "-97_ctrl"
  - "-96_ctrl"
  - "-95_ctrl"
  - "-94_ctrl"
  - "-93_ctrl"
  - "-92_ctrl"
  - "-91_ctrl"
  - "-90_ctrl"
  - "-89_ctrl"
  - "-88_ctrl"
  - "-87_ctrl"
  - "-86_ctrl"
  - "-85_ctrl"
  - "-84_ctrl"
  - "-83_ctrl"
  - "-82_ctrl"
  - "-81_ctrl"
  - "-80_ctrl"
  - "-79_ctrl"
  - "-78_ctrl"
  - "-77_ctrl"
  - "-76_ctrl"
  - "-75_ctrl"
  - "-74_ctrl"
  - "-73_ctrl"
  - "-72_ctrl"
  - "-71_ctrl"
  - "-70_ctrl"
  - "-69_ctrl"
  - "-68_ctrl"
  - "-67_ctrl"
  - "-66_ctrl"
  - "-65_ctrl"
  - "-64_ctrl"
  - "-63_ctrl"
  - "-62_ctrl"
  - "-61_ctrl"
  - "-60_ctrl"
  - "-59_ctrl"
  - "-58_ctrl"
  - "-57_ctrl"
  - "-56_ctrl"
  - "-55_ctrl"
  - "-54_ctrl"
  - "-53_ctrl"
  - "-52_ctrl"
  - "-51_ctrl"
  - "-50_ctrl"
  - "-49_ctrl"
  - "-48_ctrl"
  - "-47_ctrl"
  - "-46_ctrl"
  - "-45_ctrl"
  - "-44_ctrl"
  - "-43_ctrl"
  - "-42_ctrl"
  - "-41_ctrl"
  - "-40_ctrl"
  - "-39_ctrl"
  - "-38_ctrl"
  - "-37_ctrl"
  - "-36_ctrl"
  - "-35_ctrl"
  - "-34_ctrl"
  - "-33_ctrl"
  - "-32_ctrl"
  - "-31_ctrl"
  - "-30_ctrl"
  - "-29_ctrl"
  - "-28_ctrl"
  - "-27_ctrl"
  - "-26_ctrl"
  - "-25_ctrl"
  - "-24_ctrl"
  - "-23_ctrl"
  - "-22_ctrl"
  - "-21_ctrl"
  - "-20_ctrl"
  - "-19_ctrl"
  - "-18_ctrl"
  - "-17_ctrl"
  - "-16_ctrl"
  - "-15_ctrl"
  - "-14_ctrl"
  - "-13_ctrl"
  - "-12_ctrl"
  - "-11_ctrl"
  - "-10_ctrl"
  - "-9_ctrl"
  - "-8_ctrl"
  - "-7_ctrl"
  - "-6_ctrl"
  - "-5_ctrl"
  - "-4_ctrl"
  - "-3_ctrl"
  - "-2_ctrl"
  - "-1_ctrl"
  - "0_ctrl"
  - "1_ctrl"
  - "2_ctrl"
  - "3_ctrl"
  - "4_ctrl"
  - "5_ctrl"
  - "6_ctrl"
  - "7_ctrl"
  - "8_ctrl"
  - "9_ctrl"
  - "10_ctrl"
  - "11_ctrl"
  - "12_ctrl"
  - "13_ctrl"
  - "14_ctrl"
  - "15_ctrl"
  - "16_ctrl"
  - "17_ctrl"
  - "18_ctrl"
  - "19_ctrl"
  - "20_ctrl"
  - "21_ctrl"
  - "22_ctrl"
  - "23_ctrl"
  - "24_ctrl"
  - "25_ctrl"
  - "26_ctrl"
  - "27_ctrl"
  - "28_ctrl"
  - "29_ctrl"
  - "30_ctrl"
  - "31_ctrl"
  - "32_ctrl"
  - "33_ctrl"
  - "34_ctrl"
  - "35_ctrl"
  - "36_ctrl"
  - "37_ctrl"
  - "38_ctrl"
  - "39_ctrl"
  - "40_ctrl"
  - "41_ctrl"
  - "42_ctrl"
  - "43_ctrl"
  - "44_ctrl"
  - "45_ctrl"
  - "46_ctrl"
  - "47_ctrl"
  - "48_ctrl"
  - "49_ctrl"
  - "50_ctrl"
  - "51_ctrl"
  - "52_ctrl"
  - "53_ctrl"
  - "54_ctrl"
  - "55_ctrl"
  - "56_ctrl"
  - "57_ctrl"
  - "58_ctrl"
  - "59_ctrl"
  - "60_ctrl"
  - "61_ctrl"
  - "62_ctrl"
  - "63_ctrl"
  - "64_ctrl"
  - "65_ctrl"
  - "66_ctrl"
  - "67_ctrl"
  - "68_ctrl"
  - "69_ctrl"
  - "70_ctrl"
  - "71_ctrl"
  - "72_ctrl"
  - "73_ctrl"
  - "74_ctrl"
  - "75_ctrl"
  - "76_ctrl"
  - "77_ctrl"
  - "78_ctrl"
  - "79_ctrl"
  - "80_ctrl"
  - "81_ctrl"
  - "82_ctrl"
  - "83_ctrl"
  - "84_ctrl"
  - "85_ctrl"
  - "86_ctrl"
  - "87_ctrl"
  - "88_ctrl"
  - "89_ctrl"
  - "90_ctrl"
  - "91_ctrl"
  - "92_ctrl"
  - "93_ctrl"
  - "94_ctrl"
  - "95_ctrl"
  - "96_ctrl"
  - "97_ctrl"
  - "98_ctrl"
  - "99_ctrl"
  - "100_ctrl"
  - "101_ctrl"
  - "102_ctrl"
  - "103_ctrl"
  - "104_ctrl"
  - "105_ctrl"
  - "106_ctrl"
  - "107_ctrl"
  - "108_ctrl"
  - "109_ctrl"
  - "110_ctrl"
  - "111_ctrl"
  - "112_ctrl"
  - "113_ctrl"
  - "114_ctrl"
  - "115_ctrl"
  - "116_ctrl"
  - "117_ctrl"
  - "118_ctrl"
  - "119_ctrl"
  - "120_ctrl"
  - "121_ctrl"
  - "122_ctrl"
  - "123_ctrl"
  - "124_ctrl"
  - "125_ctrl"
  - "126_ctrl"
  - "127_ctrl"
  - "128_ctrl"
  - "129_ctrl"
  - "130_ctrl"
  - "131_ctrl"
  - "132_ctrl"
  - "133_ctrl"
  - "134_ctrl"
  - "135_ctrl"
  - "136_ctrl"
  - "137_ctrl"
  - "138_ctrl"
  - "139_ctrl"
  - "140_ctrl"
  - "141_ctrl"
  - "142_ctrl"
  - "143_ctrl"
  - "144_ctrl"
  - "145_ctrl"
  - "146_ctrl"
  - "147_ctrl"
  - "148_ctrl"
  - "149_ctrl"
  - "150_ctrl"
  - "151_ctrl"
  - "152_ctrl"
  - "153_ctrl"
  - "154_ctrl"
  - "155_ctrl"
  - "156_ctrl"
  - "157_ctrl"
  - "158_ctrl"
  - "159_ctrl"
  - "160_ctrl"
  - "161_ctrl"
  - "162_ctrl"
  - "163_ctrl"
  - "164_ctrl"
  - "165_ctrl"
  - "166_ctrl"
  - "167_ctrl"
  - "168_ctrl"
  - "169_ctrl"
  - "170_ctrl"
  - "171_ctrl"
  - "172_ctrl"
  - "173_ctrl"
  - "174_ctrl"
  - "175_ctrl"
  - "176_ctrl"
  - "177_ctrl"
  - "178_ctrl"
  - "179_ctrl"
  - "180_ctrl"
  - "181_ctrl"
  - "182_ctrl"
  - "183_ctrl"
  - "184_ctrl"
  - "185_ctrl"
  - "186_ctrl"
  - "187_ctrl"
  - "188_ctrl"
  - "189_ctrl"
  - "190_ctrl"
  - "191_ctrl"
  - "192_ctrl"
  - "193_ctrl"
  - "194_ctrl"
  - "195_ctrl"
  - "196_ctrl"
  - "197_ctrl"
  - "198_ctrl"
  - "199_ctrl"
  - "200_ctrl"
  """ }

predict_perturbations_yaml_contents = {"fusion.yaml":"""  
model_config:
  fc_do: 0.4
  fc_task_dim: 256
  layers:
  - 2
  rb_do: 0.4
  stochastic_depth_p: 0.5
model_type: "mlp-residual"
""",
"globals.yaml": """

checkpoint_interval: 30300 # 100 epochs #60000
sample_interval: 30300 #60000
n_epochs: 120
batch_size: 64
optimizer: "adamw"
lr: 0.0001 #0.0001
device: "cuda"
compute_attributions: false

""",
"input_cnn.yaml": """
input_info:
  input_source: ./inputs/perturbed_landscape_arrays/test/  #inserted_enhancer_test #inputs/silenced_arrays/silenced_arrays_H2B_S.D 
  #./data/parsed_data/inputs/arrays_train_100bp_no_H3K27ac/ #arrays_train_100bp_no_H3K27ac_uncoupled/
  input_name: gene_expression
  input_type: array

model_config:
  model_type: cnn
  #pre_normalization: "instancenorm"
  model_init_config:
    num_output_features: 512 # before fc_repr_dim
    layers: [4,4]
    kernel_height: 1
    down_stride_width: 2
    first_stride_expansion_width: 1
    first_kernel_expansion_height: 4 #5
    kernel_width: 10  #10
    dilation_factor_width: 2
    dilation_factor_height: 1
    channel_exp_base: 5  #3 #-1
    first_channel_expansion: 2
    rb_do: .3
    stochastic_depth_p: .1
    attention_inclusion_cutoff: 1 #50 #256
    """,
"outputs_2_cond.yaml": """
output_info:
  output_name: expression_output
  output_source: ./targets/perturbed_targets.csv 
  #./data/parsed_data/targets/target_arrays_perturbational_inserted_enhancer.csv #target_arrays_perturbational_S.D.csv #target_arrays_1kbp_401_bins_2_conditions_decareads_abs.csv
  output_type: tabular

model_config: # <- new
  model_type: linear # <- new

output_type_info:
  con_loss_name: "SmoothL1Loss"
  target_con_columns:
  - "-200_ctrl"
  - "-199_ctrl"
  - "-198_ctrl"
  - "-197_ctrl"
  - "-196_ctrl"
  - "-195_ctrl"
  - "-194_ctrl"
  - "-193_ctrl"
  - "-192_ctrl"
  - "-191_ctrl"
  - "-190_ctrl"
  - "-189_ctrl"
  - "-188_ctrl"
  - "-187_ctrl"
  - "-186_ctrl"
  - "-185_ctrl"
  - "-184_ctrl"
  - "-183_ctrl"
  - "-182_ctrl"
  - "-181_ctrl"
  - "-180_ctrl"
  - "-179_ctrl"
  - "-178_ctrl"
  - "-177_ctrl"
  - "-176_ctrl"
  - "-175_ctrl"
  - "-174_ctrl"
  - "-173_ctrl"
  - "-172_ctrl"
  - "-171_ctrl"
  - "-170_ctrl"
  - "-169_ctrl"
  - "-168_ctrl"
  - "-167_ctrl"
  - "-166_ctrl"
  - "-165_ctrl"
  - "-164_ctrl"
  - "-163_ctrl"
  - "-162_ctrl"
  - "-161_ctrl"
  - "-160_ctrl"
  - "-159_ctrl"
  - "-158_ctrl"
  - "-157_ctrl"
  - "-156_ctrl"
  - "-155_ctrl"
  - "-154_ctrl"
  - "-153_ctrl"
  - "-152_ctrl"
  - "-151_ctrl"
  - "-150_ctrl"
  - "-149_ctrl"
  - "-148_ctrl"
  - "-147_ctrl"
  - "-146_ctrl"
  - "-145_ctrl"
  - "-144_ctrl"
  - "-143_ctrl"
  - "-142_ctrl"
  - "-141_ctrl"
  - "-140_ctrl"
  - "-139_ctrl"
  - "-138_ctrl"
  - "-137_ctrl"
  - "-136_ctrl"
  - "-135_ctrl"
  - "-134_ctrl"
  - "-133_ctrl"
  - "-132_ctrl"
  - "-131_ctrl"
  - "-130_ctrl"
  - "-129_ctrl"
  - "-128_ctrl"
  - "-127_ctrl"
  - "-126_ctrl"
  - "-125_ctrl"
  - "-124_ctrl"
  - "-123_ctrl"
  - "-122_ctrl"
  - "-121_ctrl"
  - "-120_ctrl"
  - "-119_ctrl"
  - "-118_ctrl"
  - "-117_ctrl"
  - "-116_ctrl"
  - "-115_ctrl"
  - "-114_ctrl"
  - "-113_ctrl"
  - "-112_ctrl"
  - "-111_ctrl"
  - "-110_ctrl"
  - "-109_ctrl"
  - "-108_ctrl"
  - "-107_ctrl"
  - "-106_ctrl"
  - "-105_ctrl"
  - "-104_ctrl"
  - "-103_ctrl"
  - "-102_ctrl"
  - "-101_ctrl"
  - "-100_ctrl"
  - "-99_ctrl"
  - "-98_ctrl"
  - "-97_ctrl"
  - "-96_ctrl"
  - "-95_ctrl"
  - "-94_ctrl"
  - "-93_ctrl"
  - "-92_ctrl"
  - "-91_ctrl"
  - "-90_ctrl"
  - "-89_ctrl"
  - "-88_ctrl"
  - "-87_ctrl"
  - "-86_ctrl"
  - "-85_ctrl"
  - "-84_ctrl"
  - "-83_ctrl"
  - "-82_ctrl"
  - "-81_ctrl"
  - "-80_ctrl"
  - "-79_ctrl"
  - "-78_ctrl"
  - "-77_ctrl"
  - "-76_ctrl"
  - "-75_ctrl"
  - "-74_ctrl"
  - "-73_ctrl"
  - "-72_ctrl"
  - "-71_ctrl"
  - "-70_ctrl"
  - "-69_ctrl"
  - "-68_ctrl"
  - "-67_ctrl"
  - "-66_ctrl"
  - "-65_ctrl"
  - "-64_ctrl"
  - "-63_ctrl"
  - "-62_ctrl"
  - "-61_ctrl"
  - "-60_ctrl"
  - "-59_ctrl"
  - "-58_ctrl"
  - "-57_ctrl"
  - "-56_ctrl"
  - "-55_ctrl"
  - "-54_ctrl"
  - "-53_ctrl"
  - "-52_ctrl"
  - "-51_ctrl"
  - "-50_ctrl"
  - "-49_ctrl"
  - "-48_ctrl"
  - "-47_ctrl"
  - "-46_ctrl"
  - "-45_ctrl"
  - "-44_ctrl"
  - "-43_ctrl"
  - "-42_ctrl"
  - "-41_ctrl"
  - "-40_ctrl"
  - "-39_ctrl"
  - "-38_ctrl"
  - "-37_ctrl"
  - "-36_ctrl"
  - "-35_ctrl"
  - "-34_ctrl"
  - "-33_ctrl"
  - "-32_ctrl"
  - "-31_ctrl"
  - "-30_ctrl"
  - "-29_ctrl"
  - "-28_ctrl"
  - "-27_ctrl"
  - "-26_ctrl"
  - "-25_ctrl"
  - "-24_ctrl"
  - "-23_ctrl"
  - "-22_ctrl"
  - "-21_ctrl"
  - "-20_ctrl"
  - "-19_ctrl"
  - "-18_ctrl"
  - "-17_ctrl"
  - "-16_ctrl"
  - "-15_ctrl"
  - "-14_ctrl"
  - "-13_ctrl"
  - "-12_ctrl"
  - "-11_ctrl"
  - "-10_ctrl"
  - "-9_ctrl"
  - "-8_ctrl"
  - "-7_ctrl"
  - "-6_ctrl"
  - "-5_ctrl"
  - "-4_ctrl"
  - "-3_ctrl"
  - "-2_ctrl"
  - "-1_ctrl"
  - "0_ctrl"
  - "1_ctrl"
  - "2_ctrl"
  - "3_ctrl"
  - "4_ctrl"
  - "5_ctrl"
  - "6_ctrl"
  - "7_ctrl"
  - "8_ctrl"
  - "9_ctrl"
  - "10_ctrl"
  - "11_ctrl"
  - "12_ctrl"
  - "13_ctrl"
  - "14_ctrl"
  - "15_ctrl"
  - "16_ctrl"
  - "17_ctrl"
  - "18_ctrl"
  - "19_ctrl"
  - "20_ctrl"
  - "21_ctrl"
  - "22_ctrl"
  - "23_ctrl"
  - "24_ctrl"
  - "25_ctrl"
  - "26_ctrl"
  - "27_ctrl"
  - "28_ctrl"
  - "29_ctrl"
  - "30_ctrl"
  - "31_ctrl"
  - "32_ctrl"
  - "33_ctrl"
  - "34_ctrl"
  - "35_ctrl"
  - "36_ctrl"
  - "37_ctrl"
  - "38_ctrl"
  - "39_ctrl"
  - "40_ctrl"
  - "41_ctrl"
  - "42_ctrl"
  - "43_ctrl"
  - "44_ctrl"
  - "45_ctrl"
  - "46_ctrl"
  - "47_ctrl"
  - "48_ctrl"
  - "49_ctrl"
  - "50_ctrl"
  - "51_ctrl"
  - "52_ctrl"
  - "53_ctrl"
  - "54_ctrl"
  - "55_ctrl"
  - "56_ctrl"
  - "57_ctrl"
  - "58_ctrl"
  - "59_ctrl"
  - "60_ctrl"
  - "61_ctrl"
  - "62_ctrl"
  - "63_ctrl"
  - "64_ctrl"
  - "65_ctrl"
  - "66_ctrl"
  - "67_ctrl"
  - "68_ctrl"
  - "69_ctrl"
  - "70_ctrl"
  - "71_ctrl"
  - "72_ctrl"
  - "73_ctrl"
  - "74_ctrl"
  - "75_ctrl"
  - "76_ctrl"
  - "77_ctrl"
  - "78_ctrl"
  - "79_ctrl"
  - "80_ctrl"
  - "81_ctrl"
  - "82_ctrl"
  - "83_ctrl"
  - "84_ctrl"
  - "85_ctrl"
  - "86_ctrl"
  - "87_ctrl"
  - "88_ctrl"
  - "89_ctrl"
  - "90_ctrl"
  - "91_ctrl"
  - "92_ctrl"
  - "93_ctrl"
  - "94_ctrl"
  - "95_ctrl"
  - "96_ctrl"
  - "97_ctrl"
  - "98_ctrl"
  - "99_ctrl"
  - "100_ctrl"
  - "101_ctrl"
  - "102_ctrl"
  - "103_ctrl"
  - "104_ctrl"
  - "105_ctrl"
  - "106_ctrl"
  - "107_ctrl"
  - "108_ctrl"
  - "109_ctrl"
  - "110_ctrl"
  - "111_ctrl"
  - "112_ctrl"
  - "113_ctrl"
  - "114_ctrl"
  - "115_ctrl"
  - "116_ctrl"
  - "117_ctrl"
  - "118_ctrl"
  - "119_ctrl"
  - "120_ctrl"
  - "121_ctrl"
  - "122_ctrl"
  - "123_ctrl"
  - "124_ctrl"
  - "125_ctrl"
  - "126_ctrl"
  - "127_ctrl"
  - "128_ctrl"
  - "129_ctrl"
  - "130_ctrl"
  - "131_ctrl"
  - "132_ctrl"
  - "133_ctrl"
  - "134_ctrl"
  - "135_ctrl"
  - "136_ctrl"
  - "137_ctrl"
  - "138_ctrl"
  - "139_ctrl"
  - "140_ctrl"
  - "141_ctrl"
  - "142_ctrl"
  - "143_ctrl"
  - "144_ctrl"
  - "145_ctrl"
  - "146_ctrl"
  - "147_ctrl"
  - "148_ctrl"
  - "149_ctrl"
  - "150_ctrl"
  - "151_ctrl"
  - "152_ctrl"
  - "153_ctrl"
  - "154_ctrl"
  - "155_ctrl"
  - "156_ctrl"
  - "157_ctrl"
  - "158_ctrl"
  - "159_ctrl"
  - "160_ctrl"
  - "161_ctrl"
  - "162_ctrl"
  - "163_ctrl"
  - "164_ctrl"
  - "165_ctrl"
  - "166_ctrl"
  - "167_ctrl"
  - "168_ctrl"
  - "169_ctrl"
  - "170_ctrl"
  - "171_ctrl"
  - "172_ctrl"
  - "173_ctrl"
  - "174_ctrl"
  - "175_ctrl"
  - "176_ctrl"
  - "177_ctrl"
  - "178_ctrl"
  - "179_ctrl"
  - "180_ctrl"
  - "181_ctrl"
  - "182_ctrl"
  - "183_ctrl"
  - "184_ctrl"
  - "185_ctrl"
  - "186_ctrl"
  - "187_ctrl"
  - "188_ctrl"
  - "189_ctrl"
  - "190_ctrl"
  - "191_ctrl"
  - "192_ctrl"
  - "193_ctrl"
  - "194_ctrl"
  - "195_ctrl"
  - "196_ctrl"
  - "197_ctrl"
  - "198_ctrl"
  - "199_ctrl"
  - "200_ctrl"
  """ }

training_microc_yaml_contents = {"globals.yaml":"""
output_folder: ./runs/gene_expression_microc_pure_conv/  #gene_expression_prediction_no_H3K27ac_uncoupled/     #gene_expression_cnn_1kbp_401_bins_2_cond_reloaded/ 
manual_valid_ids_file: ./annotations/manual_validation_ids_chr17.txt  #manual_validation_ids_chr17_uncoupled.txt
checkpoint_interval: 30300 
sample_interval: 30300 
n_epochs: 120
batch_size: 64
optimizer: "adamw"
lr: 0.0001 
device: "cuda"
compute_attributions: false
# latent_sampling: 
#   layers_to_sample:
#     - "input_modules.contact_maps.feature_extractor.conv.0.conv_1"
#     - "input_modules.contact_maps.feature_extractor.conv.1.conv_2"
#     - "input_modules.contact_maps.feature_extractor.conv.1.conv_2"
""",
"input_cnn.yaml": """ 
input_info:
  input_source: ./inputs/landscape_arrays/training/ 
  input_name: gene_expression
  input_type: array

model_config:
  model_type: cnn
  #pre_normalization: "instancenorm"
  model_init_config:
    num_output_features: 512 # before fc_repr_dim
    layers: [4,4]
    kernel_height: 1
    down_stride_width: 2
    first_stride_expansion_width: 1
    first_kernel_expansion_height: 4 #5
    kernel_width: 10  #10
    dilation_factor_width: 2
    dilation_factor_height: 1
    channel_exp_base: 5  #3 #-1
    first_channel_expansion: 2
    rb_do: .3
    stochastic_depth_p: .1
    attention_inclusion_cutoff: 1 #50 #256

""",
"""input_cnn_microc.yaml""": """
input_info:
  input_source: ./inputs/microC/training/ 
  input_name: contact_maps
  input_type: array

model_config:
  model_type: cnn
  #pre_normalization: "instancenorm"
  model_init_config:
    num_output_features: 512 #before fc_repr_dim
    layers: [1,2]
    kernel_height: 5
    down_stride_width: 2
    down_stride_height: 2 #5
    kernel_width: 5  #10
    dilation_factor_width: 2
    dilation_factor_height: 2
    channel_exp_base: 2  #3 #-1
    first_channel_expansion: 2
    rb_do: .3
    stochastic_depth_p: .1
    attention_inclusion_cutoff: 1 #128 #256
""",
"fusion.yaml": """
model_config:
  fc_do: 0.4
  fc_task_dim: 256
  layers:
  - 2
  rb_do: 0.4
  stochastic_depth_p: 0.5
model_type: "mlp-residual"
""",
"outputs_2_cond.yaml":"""
output_info:
  output_name: expression_output
  output_source: ./targets/training_targets.csv 
  output_type: tabular

model_config: # <- new
  model_type: linear # <- new

output_type_info:
  con_loss_name: "SmoothL1Loss"
  target_con_columns:
  - "-200_ctrl"
  - "-199_ctrl"
  - "-198_ctrl"
  - "-197_ctrl"
  - "-196_ctrl"
  - "-195_ctrl"
  - "-194_ctrl"
  - "-193_ctrl"
  - "-192_ctrl"
  - "-191_ctrl"
  - "-190_ctrl"
  - "-189_ctrl"
  - "-188_ctrl"
  - "-187_ctrl"
  - "-186_ctrl"
  - "-185_ctrl"
  - "-184_ctrl"
  - "-183_ctrl"
  - "-182_ctrl"
  - "-181_ctrl"
  - "-180_ctrl"
  - "-179_ctrl"
  - "-178_ctrl"
  - "-177_ctrl"
  - "-176_ctrl"
  - "-175_ctrl"
  - "-174_ctrl"
  - "-173_ctrl"
  - "-172_ctrl"
  - "-171_ctrl"
  - "-170_ctrl"
  - "-169_ctrl"
  - "-168_ctrl"
  - "-167_ctrl"
  - "-166_ctrl"
  - "-165_ctrl"
  - "-164_ctrl"
  - "-163_ctrl"
  - "-162_ctrl"
  - "-161_ctrl"
  - "-160_ctrl"
  - "-159_ctrl"
  - "-158_ctrl"
  - "-157_ctrl"
  - "-156_ctrl"
  - "-155_ctrl"
  - "-154_ctrl"
  - "-153_ctrl"
  - "-152_ctrl"
  - "-151_ctrl"
  - "-150_ctrl"
  - "-149_ctrl"
  - "-148_ctrl"
  - "-147_ctrl"
  - "-146_ctrl"
  - "-145_ctrl"
  - "-144_ctrl"
  - "-143_ctrl"
  - "-142_ctrl"
  - "-141_ctrl"
  - "-140_ctrl"
  - "-139_ctrl"
  - "-138_ctrl"
  - "-137_ctrl"
  - "-136_ctrl"
  - "-135_ctrl"
  - "-134_ctrl"
  - "-133_ctrl"
  - "-132_ctrl"
  - "-131_ctrl"
  - "-130_ctrl"
  - "-129_ctrl"
  - "-128_ctrl"
  - "-127_ctrl"
  - "-126_ctrl"
  - "-125_ctrl"
  - "-124_ctrl"
  - "-123_ctrl"
  - "-122_ctrl"
  - "-121_ctrl"
  - "-120_ctrl"
  - "-119_ctrl"
  - "-118_ctrl"
  - "-117_ctrl"
  - "-116_ctrl"
  - "-115_ctrl"
  - "-114_ctrl"
  - "-113_ctrl"
  - "-112_ctrl"
  - "-111_ctrl"
  - "-110_ctrl"
  - "-109_ctrl"
  - "-108_ctrl"
  - "-107_ctrl"
  - "-106_ctrl"
  - "-105_ctrl"
  - "-104_ctrl"
  - "-103_ctrl"
  - "-102_ctrl"
  - "-101_ctrl"
  - "-100_ctrl"
  - "-99_ctrl"
  - "-98_ctrl"
  - "-97_ctrl"
  - "-96_ctrl"
  - "-95_ctrl"
  - "-94_ctrl"
  - "-93_ctrl"
  - "-92_ctrl"
  - "-91_ctrl"
  - "-90_ctrl"
  - "-89_ctrl"
  - "-88_ctrl"
  - "-87_ctrl"
  - "-86_ctrl"
  - "-85_ctrl"
  - "-84_ctrl"
  - "-83_ctrl"
  - "-82_ctrl"
  - "-81_ctrl"
  - "-80_ctrl"
  - "-79_ctrl"
  - "-78_ctrl"
  - "-77_ctrl"
  - "-76_ctrl"
  - "-75_ctrl"
  - "-74_ctrl"
  - "-73_ctrl"
  - "-72_ctrl"
  - "-71_ctrl"
  - "-70_ctrl"
  - "-69_ctrl"
  - "-68_ctrl"
  - "-67_ctrl"
  - "-66_ctrl"
  - "-65_ctrl"
  - "-64_ctrl"
  - "-63_ctrl"
  - "-62_ctrl"
  - "-61_ctrl"
  - "-60_ctrl"
  - "-59_ctrl"
  - "-58_ctrl"
  - "-57_ctrl"
  - "-56_ctrl"
  - "-55_ctrl"
  - "-54_ctrl"
  - "-53_ctrl"
  - "-52_ctrl"
  - "-51_ctrl"
  - "-50_ctrl"
  - "-49_ctrl"
  - "-48_ctrl"
  - "-47_ctrl"
  - "-46_ctrl"
  - "-45_ctrl"
  - "-44_ctrl"
  - "-43_ctrl"
  - "-42_ctrl"
  - "-41_ctrl"
  - "-40_ctrl"
  - "-39_ctrl"
  - "-38_ctrl"
  - "-37_ctrl"
  - "-36_ctrl"
  - "-35_ctrl"
  - "-34_ctrl"
  - "-33_ctrl"
  - "-32_ctrl"
  - "-31_ctrl"
  - "-30_ctrl"
  - "-29_ctrl"
  - "-28_ctrl"
  - "-27_ctrl"
  - "-26_ctrl"
  - "-25_ctrl"
  - "-24_ctrl"
  - "-23_ctrl"
  - "-22_ctrl"
  - "-21_ctrl"
  - "-20_ctrl"
  - "-19_ctrl"
  - "-18_ctrl"
  - "-17_ctrl"
  - "-16_ctrl"
  - "-15_ctrl"
  - "-14_ctrl"
  - "-13_ctrl"
  - "-12_ctrl"
  - "-11_ctrl"
  - "-10_ctrl"
  - "-9_ctrl"
  - "-8_ctrl"
  - "-7_ctrl"
  - "-6_ctrl"
  - "-5_ctrl"
  - "-4_ctrl"
  - "-3_ctrl"
  - "-2_ctrl"
  - "-1_ctrl"
  - "0_ctrl"
  - "1_ctrl"
  - "2_ctrl"
  - "3_ctrl"
  - "4_ctrl"
  - "5_ctrl"
  - "6_ctrl"
  - "7_ctrl"
  - "8_ctrl"
  - "9_ctrl"
  - "10_ctrl"
  - "11_ctrl"
  - "12_ctrl"
  - "13_ctrl"
  - "14_ctrl"
  - "15_ctrl"
  - "16_ctrl"
  - "17_ctrl"
  - "18_ctrl"
  - "19_ctrl"
  - "20_ctrl"
  - "21_ctrl"
  - "22_ctrl"
  - "23_ctrl"
  - "24_ctrl"
  - "25_ctrl"
  - "26_ctrl"
  - "27_ctrl"
  - "28_ctrl"
  - "29_ctrl"
  - "30_ctrl"
  - "31_ctrl"
  - "32_ctrl"
  - "33_ctrl"
  - "34_ctrl"
  - "35_ctrl"
  - "36_ctrl"
  - "37_ctrl"
  - "38_ctrl"
  - "39_ctrl"
  - "40_ctrl"
  - "41_ctrl"
  - "42_ctrl"
  - "43_ctrl"
  - "44_ctrl"
  - "45_ctrl"
  - "46_ctrl"
  - "47_ctrl"
  - "48_ctrl"
  - "49_ctrl"
  - "50_ctrl"
  - "51_ctrl"
  - "52_ctrl"
  - "53_ctrl"
  - "54_ctrl"
  - "55_ctrl"
  - "56_ctrl"
  - "57_ctrl"
  - "58_ctrl"
  - "59_ctrl"
  - "60_ctrl"
  - "61_ctrl"
  - "62_ctrl"
  - "63_ctrl"
  - "64_ctrl"
  - "65_ctrl"
  - "66_ctrl"
  - "67_ctrl"
  - "68_ctrl"
  - "69_ctrl"
  - "70_ctrl"
  - "71_ctrl"
  - "72_ctrl"
  - "73_ctrl"
  - "74_ctrl"
  - "75_ctrl"
  - "76_ctrl"
  - "77_ctrl"
  - "78_ctrl"
  - "79_ctrl"
  - "80_ctrl"
  - "81_ctrl"
  - "82_ctrl"
  - "83_ctrl"
  - "84_ctrl"
  - "85_ctrl"
  - "86_ctrl"
  - "87_ctrl"
  - "88_ctrl"
  - "89_ctrl"
  - "90_ctrl"
  - "91_ctrl"
  - "92_ctrl"
  - "93_ctrl"
  - "94_ctrl"
  - "95_ctrl"
  - "96_ctrl"
  - "97_ctrl"
  - "98_ctrl"
  - "99_ctrl"
  - "100_ctrl"
  - "101_ctrl"
  - "102_ctrl"
  - "103_ctrl"
  - "104_ctrl"
  - "105_ctrl"
  - "106_ctrl"
  - "107_ctrl"
  - "108_ctrl"
  - "109_ctrl"
  - "110_ctrl"
  - "111_ctrl"
  - "112_ctrl"
  - "113_ctrl"
  - "114_ctrl"
  - "115_ctrl"
  - "116_ctrl"
  - "117_ctrl"
  - "118_ctrl"
  - "119_ctrl"
  - "120_ctrl"
  - "121_ctrl"
  - "122_ctrl"
  - "123_ctrl"
  - "124_ctrl"
  - "125_ctrl"
  - "126_ctrl"
  - "127_ctrl"
  - "128_ctrl"
  - "129_ctrl"
  - "130_ctrl"
  - "131_ctrl"
  - "132_ctrl"
  - "133_ctrl"
  - "134_ctrl"
  - "135_ctrl"
  - "136_ctrl"
  - "137_ctrl"
  - "138_ctrl"
  - "139_ctrl"
  - "140_ctrl"
  - "141_ctrl"
  - "142_ctrl"
  - "143_ctrl"
  - "144_ctrl"
  - "145_ctrl"
  - "146_ctrl"
  - "147_ctrl"
  - "148_ctrl"
  - "149_ctrl"
  - "150_ctrl"
  - "151_ctrl"
  - "152_ctrl"
  - "153_ctrl"
  - "154_ctrl"
  - "155_ctrl"
  - "156_ctrl"
  - "157_ctrl"
  - "158_ctrl"
  - "159_ctrl"
  - "160_ctrl"
  - "161_ctrl"
  - "162_ctrl"
  - "163_ctrl"
  - "164_ctrl"
  - "165_ctrl"
  - "166_ctrl"
  - "167_ctrl"
  - "168_ctrl"
  - "169_ctrl"
  - "170_ctrl"
  - "171_ctrl"
  - "172_ctrl"
  - "173_ctrl"
  - "174_ctrl"
  - "175_ctrl"
  - "176_ctrl"
  - "177_ctrl"
  - "178_ctrl"
  - "179_ctrl"
  - "180_ctrl"
  - "181_ctrl"
  - "182_ctrl"
  - "183_ctrl"
  - "184_ctrl"
  - "185_ctrl"
  - "186_ctrl"
  - "187_ctrl"
  - "188_ctrl"
  - "189_ctrl"
  - "190_ctrl"
  - "191_ctrl"
  - "192_ctrl"
  - "193_ctrl"
  - "194_ctrl"
  - "195_ctrl"
  - "196_ctrl"
  - "197_ctrl"
  - "198_ctrl"
  - "199_ctrl"
  - "200_ctrl"
  """
}

test_microc_yaml_contents = {"globals.yaml":"""
#output_folder: ./runs/gene_expression_microc_pure_conv/  #gene_expression_prediction_no_H3K27ac_uncoupled/     #gene_expression_cnn_1kbp_401_bins_2_cond_reloaded/ 
#manual_valid_ids_file: ./annotations/manual_validation_ids_chr17.txt  #manual_validation_ids_chr17_uncoupled.txt
checkpoint_interval: 30300 
sample_interval: 30300 
n_epochs: 120
batch_size: 64
optimizer: "adamw"
lr: 0.0001 
device: "cuda"
compute_attributions: false
# latent_sampling: 
#   layers_to_sample:
#     - "input_modules.contact_maps.feature_extractor.conv.0.conv_1"
#     - "input_modules.contact_maps.feature_extractor.conv.1.conv_1"
#     - "input_modules.contact_maps.feature_extractor.conv.1.conv_2"
""",
"input_cnn.yaml": """ 
input_info:
  input_source: ./inputs/landscape_arrays/test/ 
  input_name: gene_expression
  input_type: array

model_config:
  model_type: cnn
  #pre_normalization: "instancenorm"
  model_init_config:
    num_output_features: 512 # before fc_repr_dim
    layers: [4,4]
    kernel_height: 1
    down_stride_width: 2
    first_stride_expansion_width: 1
    first_kernel_expansion_height: 4 #5
    kernel_width: 10  #10
    dilation_factor_width: 2
    dilation_factor_height: 1
    channel_exp_base: 5  #3 #-1
    first_channel_expansion: 2
    rb_do: .3
    stochastic_depth_p: .1
    attention_inclusion_cutoff: 1 #50 #256

""",
"""input_cnn_microc.yaml""": """
input_info:
  input_source: ./inputs/microC/test/ 
  input_name: contact_maps
  input_type: array

model_config:
  model_type: cnn
  #pre_normalization: "instancenorm"
  model_init_config:
    num_output_features: 512 #before fc_repr_dim
    layers: [1,2]
    kernel_height: 5
    down_stride_width: 2
    down_stride_height: 2 #5
    kernel_width: 5  #10
    dilation_factor_width: 2
    dilation_factor_height: 2
    channel_exp_base: 2  #3 #-1
    first_channel_expansion: 2
    rb_do: .3
    stochastic_depth_p: .1
    attention_inclusion_cutoff: 1 #128 #256
""",
"fusion.yaml": """
model_config:
  fc_do: 0.4
  fc_task_dim: 256
  layers:
  - 2
  rb_do: 0.4
  stochastic_depth_p: 0.5
model_type: "mlp-residual"
""",
"outputs_2_cond.yaml":"""
output_info:
  output_name: expression_output
  output_source: ./targets/test_targets.csv 
  output_type: tabular

model_config: # <- new
  model_type: linear # <- new

output_type_info:
  con_loss_name: "SmoothL1Loss"
  target_con_columns:
  - "-200_ctrl"
  - "-199_ctrl"
  - "-198_ctrl"
  - "-197_ctrl"
  - "-196_ctrl"
  - "-195_ctrl"
  - "-194_ctrl"
  - "-193_ctrl"
  - "-192_ctrl"
  - "-191_ctrl"
  - "-190_ctrl"
  - "-189_ctrl"
  - "-188_ctrl"
  - "-187_ctrl"
  - "-186_ctrl"
  - "-185_ctrl"
  - "-184_ctrl"
  - "-183_ctrl"
  - "-182_ctrl"
  - "-181_ctrl"
  - "-180_ctrl"
  - "-179_ctrl"
  - "-178_ctrl"
  - "-177_ctrl"
  - "-176_ctrl"
  - "-175_ctrl"
  - "-174_ctrl"
  - "-173_ctrl"
  - "-172_ctrl"
  - "-171_ctrl"
  - "-170_ctrl"
  - "-169_ctrl"
  - "-168_ctrl"
  - "-167_ctrl"
  - "-166_ctrl"
  - "-165_ctrl"
  - "-164_ctrl"
  - "-163_ctrl"
  - "-162_ctrl"
  - "-161_ctrl"
  - "-160_ctrl"
  - "-159_ctrl"
  - "-158_ctrl"
  - "-157_ctrl"
  - "-156_ctrl"
  - "-155_ctrl"
  - "-154_ctrl"
  - "-153_ctrl"
  - "-152_ctrl"
  - "-151_ctrl"
  - "-150_ctrl"
  - "-149_ctrl"
  - "-148_ctrl"
  - "-147_ctrl"
  - "-146_ctrl"
  - "-145_ctrl"
  - "-144_ctrl"
  - "-143_ctrl"
  - "-142_ctrl"
  - "-141_ctrl"
  - "-140_ctrl"
  - "-139_ctrl"
  - "-138_ctrl"
  - "-137_ctrl"
  - "-136_ctrl"
  - "-135_ctrl"
  - "-134_ctrl"
  - "-133_ctrl"
  - "-132_ctrl"
  - "-131_ctrl"
  - "-130_ctrl"
  - "-129_ctrl"
  - "-128_ctrl"
  - "-127_ctrl"
  - "-126_ctrl"
  - "-125_ctrl"
  - "-124_ctrl"
  - "-123_ctrl"
  - "-122_ctrl"
  - "-121_ctrl"
  - "-120_ctrl"
  - "-119_ctrl"
  - "-118_ctrl"
  - "-117_ctrl"
  - "-116_ctrl"
  - "-115_ctrl"
  - "-114_ctrl"
  - "-113_ctrl"
  - "-112_ctrl"
  - "-111_ctrl"
  - "-110_ctrl"
  - "-109_ctrl"
  - "-108_ctrl"
  - "-107_ctrl"
  - "-106_ctrl"
  - "-105_ctrl"
  - "-104_ctrl"
  - "-103_ctrl"
  - "-102_ctrl"
  - "-101_ctrl"
  - "-100_ctrl"
  - "-99_ctrl"
  - "-98_ctrl"
  - "-97_ctrl"
  - "-96_ctrl"
  - "-95_ctrl"
  - "-94_ctrl"
  - "-93_ctrl"
  - "-92_ctrl"
  - "-91_ctrl"
  - "-90_ctrl"
  - "-89_ctrl"
  - "-88_ctrl"
  - "-87_ctrl"
  - "-86_ctrl"
  - "-85_ctrl"
  - "-84_ctrl"
  - "-83_ctrl"
  - "-82_ctrl"
  - "-81_ctrl"
  - "-80_ctrl"
  - "-79_ctrl"
  - "-78_ctrl"
  - "-77_ctrl"
  - "-76_ctrl"
  - "-75_ctrl"
  - "-74_ctrl"
  - "-73_ctrl"
  - "-72_ctrl"
  - "-71_ctrl"
  - "-70_ctrl"
  - "-69_ctrl"
  - "-68_ctrl"
  - "-67_ctrl"
  - "-66_ctrl"
  - "-65_ctrl"
  - "-64_ctrl"
  - "-63_ctrl"
  - "-62_ctrl"
  - "-61_ctrl"
  - "-60_ctrl"
  - "-59_ctrl"
  - "-58_ctrl"
  - "-57_ctrl"
  - "-56_ctrl"
  - "-55_ctrl"
  - "-54_ctrl"
  - "-53_ctrl"
  - "-52_ctrl"
  - "-51_ctrl"
  - "-50_ctrl"
  - "-49_ctrl"
  - "-48_ctrl"
  - "-47_ctrl"
  - "-46_ctrl"
  - "-45_ctrl"
  - "-44_ctrl"
  - "-43_ctrl"
  - "-42_ctrl"
  - "-41_ctrl"
  - "-40_ctrl"
  - "-39_ctrl"
  - "-38_ctrl"
  - "-37_ctrl"
  - "-36_ctrl"
  - "-35_ctrl"
  - "-34_ctrl"
  - "-33_ctrl"
  - "-32_ctrl"
  - "-31_ctrl"
  - "-30_ctrl"
  - "-29_ctrl"
  - "-28_ctrl"
  - "-27_ctrl"
  - "-26_ctrl"
  - "-25_ctrl"
  - "-24_ctrl"
  - "-23_ctrl"
  - "-22_ctrl"
  - "-21_ctrl"
  - "-20_ctrl"
  - "-19_ctrl"
  - "-18_ctrl"
  - "-17_ctrl"
  - "-16_ctrl"
  - "-15_ctrl"
  - "-14_ctrl"
  - "-13_ctrl"
  - "-12_ctrl"
  - "-11_ctrl"
  - "-10_ctrl"
  - "-9_ctrl"
  - "-8_ctrl"
  - "-7_ctrl"
  - "-6_ctrl"
  - "-5_ctrl"
  - "-4_ctrl"
  - "-3_ctrl"
  - "-2_ctrl"
  - "-1_ctrl"
  - "0_ctrl"
  - "1_ctrl"
  - "2_ctrl"
  - "3_ctrl"
  - "4_ctrl"
  - "5_ctrl"
  - "6_ctrl"
  - "7_ctrl"
  - "8_ctrl"
  - "9_ctrl"
  - "10_ctrl"
  - "11_ctrl"
  - "12_ctrl"
  - "13_ctrl"
  - "14_ctrl"
  - "15_ctrl"
  - "16_ctrl"
  - "17_ctrl"
  - "18_ctrl"
  - "19_ctrl"
  - "20_ctrl"
  - "21_ctrl"
  - "22_ctrl"
  - "23_ctrl"
  - "24_ctrl"
  - "25_ctrl"
  - "26_ctrl"
  - "27_ctrl"
  - "28_ctrl"
  - "29_ctrl"
  - "30_ctrl"
  - "31_ctrl"
  - "32_ctrl"
  - "33_ctrl"
  - "34_ctrl"
  - "35_ctrl"
  - "36_ctrl"
  - "37_ctrl"
  - "38_ctrl"
  - "39_ctrl"
  - "40_ctrl"
  - "41_ctrl"
  - "42_ctrl"
  - "43_ctrl"
  - "44_ctrl"
  - "45_ctrl"
  - "46_ctrl"
  - "47_ctrl"
  - "48_ctrl"
  - "49_ctrl"
  - "50_ctrl"
  - "51_ctrl"
  - "52_ctrl"
  - "53_ctrl"
  - "54_ctrl"
  - "55_ctrl"
  - "56_ctrl"
  - "57_ctrl"
  - "58_ctrl"
  - "59_ctrl"
  - "60_ctrl"
  - "61_ctrl"
  - "62_ctrl"
  - "63_ctrl"
  - "64_ctrl"
  - "65_ctrl"
  - "66_ctrl"
  - "67_ctrl"
  - "68_ctrl"
  - "69_ctrl"
  - "70_ctrl"
  - "71_ctrl"
  - "72_ctrl"
  - "73_ctrl"
  - "74_ctrl"
  - "75_ctrl"
  - "76_ctrl"
  - "77_ctrl"
  - "78_ctrl"
  - "79_ctrl"
  - "80_ctrl"
  - "81_ctrl"
  - "82_ctrl"
  - "83_ctrl"
  - "84_ctrl"
  - "85_ctrl"
  - "86_ctrl"
  - "87_ctrl"
  - "88_ctrl"
  - "89_ctrl"
  - "90_ctrl"
  - "91_ctrl"
  - "92_ctrl"
  - "93_ctrl"
  - "94_ctrl"
  - "95_ctrl"
  - "96_ctrl"
  - "97_ctrl"
  - "98_ctrl"
  - "99_ctrl"
  - "100_ctrl"
  - "101_ctrl"
  - "102_ctrl"
  - "103_ctrl"
  - "104_ctrl"
  - "105_ctrl"
  - "106_ctrl"
  - "107_ctrl"
  - "108_ctrl"
  - "109_ctrl"
  - "110_ctrl"
  - "111_ctrl"
  - "112_ctrl"
  - "113_ctrl"
  - "114_ctrl"
  - "115_ctrl"
  - "116_ctrl"
  - "117_ctrl"
  - "118_ctrl"
  - "119_ctrl"
  - "120_ctrl"
  - "121_ctrl"
  - "122_ctrl"
  - "123_ctrl"
  - "124_ctrl"
  - "125_ctrl"
  - "126_ctrl"
  - "127_ctrl"
  - "128_ctrl"
  - "129_ctrl"
  - "130_ctrl"
  - "131_ctrl"
  - "132_ctrl"
  - "133_ctrl"
  - "134_ctrl"
  - "135_ctrl"
  - "136_ctrl"
  - "137_ctrl"
  - "138_ctrl"
  - "139_ctrl"
  - "140_ctrl"
  - "141_ctrl"
  - "142_ctrl"
  - "143_ctrl"
  - "144_ctrl"
  - "145_ctrl"
  - "146_ctrl"
  - "147_ctrl"
  - "148_ctrl"
  - "149_ctrl"
  - "150_ctrl"
  - "151_ctrl"
  - "152_ctrl"
  - "153_ctrl"
  - "154_ctrl"
  - "155_ctrl"
  - "156_ctrl"
  - "157_ctrl"
  - "158_ctrl"
  - "159_ctrl"
  - "160_ctrl"
  - "161_ctrl"
  - "162_ctrl"
  - "163_ctrl"
  - "164_ctrl"
  - "165_ctrl"
  - "166_ctrl"
  - "167_ctrl"
  - "168_ctrl"
  - "169_ctrl"
  - "170_ctrl"
  - "171_ctrl"
  - "172_ctrl"
  - "173_ctrl"
  - "174_ctrl"
  - "175_ctrl"
  - "176_ctrl"
  - "177_ctrl"
  - "178_ctrl"
  - "179_ctrl"
  - "180_ctrl"
  - "181_ctrl"
  - "182_ctrl"
  - "183_ctrl"
  - "184_ctrl"
  - "185_ctrl"
  - "186_ctrl"
  - "187_ctrl"
  - "188_ctrl"
  - "189_ctrl"
  - "190_ctrl"
  - "191_ctrl"
  - "192_ctrl"
  - "193_ctrl"
  - "194_ctrl"
  - "195_ctrl"
  - "196_ctrl"
  - "197_ctrl"
  - "198_ctrl"
  - "199_ctrl"
  - "200_ctrl"
  """
}


training_microc_rotated_yaml_contents = {"globals.yaml":"""
output_folder: ./runs/gene_expression_microc_rotated_pure_conv/  #gene_expression_prediction_no_H3K27ac_uncoupled/     #gene_expression_cnn_1kbp_401_bins_2_cond_reloaded/ 
manual_valid_ids_file: ./annotations/manual_validation_ids_chr17.txt  #manual_validation_ids_chr17_uncoupled.txt
checkpoint_interval: 30300 
sample_interval: 30300 
n_epochs: 120
batch_size: 64
optimizer: "adamw"
lr: 0.0001 
device: "cuda"
compute_attributions: false
# latent_sampling: 
#   layers_to_sample:
#     - "input_modules.contact_maps.feature_extractor.conv.0.conv_1"
#     - "input_modules.contact_maps.feature_extractor.conv.1.conv_2"
#     - "input_modules.contact_maps.feature_extractor.conv.1.conv_2"
""",
"input_cnn.yaml": """ 
input_info:
  input_source: ./inputs/landscape_arrays/training/ 
  input_name: gene_expression
  input_type: array

model_config:
  model_type: cnn
  #pre_normalization: "instancenorm"
  model_init_config:
    num_output_features: 512 # before fc_repr_dim
    layers: [4,4]
    kernel_height: 1
    down_stride_width: 2
    first_stride_expansion_width: 1
    first_kernel_expansion_height: 4 #5
    kernel_width: 10  #10
    dilation_factor_width: 2
    dilation_factor_height: 1
    channel_exp_base: 5  #3 #-1
    first_channel_expansion: 2
    rb_do: .3
    stochastic_depth_p: .1
    attention_inclusion_cutoff: 1 #50 #256

""",
"""input_cnn_microc_rotated.yaml""": """ 
input_info:
  input_source: ./inputs/microC_rotated/training/ #arrays_train_100bp_no_H3K27ac_uncoupled/
  input_name: contact_maps
  input_type: array

model_config:
  model_type: cnn
  #pre_normalization: "instancenorm"
  model_init_config:
    num_output_features: 512 #before fc_repr_dim
    layers: [1,2]
    kernel_height: 1
    down_stride_width: 2
    first_stride_expansion_width: 1
    first_kernel_expansion_height: 129 #Height of the array
    kernel_width: 10  #10
    dilation_factor_width: 2
    channel_exp_base: 4  #3 #-1
    first_channel_expansion: 2
    rb_do: .3
    stochastic_depth_p: .1
    attention_inclusion_cutoff: 1 # 50 #128 #256
""",
"fusion.yaml": """
model_config:
  fc_do: 0.4
  fc_task_dim: 256
  layers:
  - 2
  rb_do: 0.4
  stochastic_depth_p: 0.5
model_type: "mlp-residual"
""",
"outputs_2_cond.yaml":"""
output_info:
  output_name: expression_output
  output_source: ./targets/training_targets.csv 
  output_type: tabular

model_config: # <- new
  model_type: linear # <- new

output_type_info:
  con_loss_name: "SmoothL1Loss"
  target_con_columns:
  - "-200_ctrl"
  - "-199_ctrl"
  - "-198_ctrl"
  - "-197_ctrl"
  - "-196_ctrl"
  - "-195_ctrl"
  - "-194_ctrl"
  - "-193_ctrl"
  - "-192_ctrl"
  - "-191_ctrl"
  - "-190_ctrl"
  - "-189_ctrl"
  - "-188_ctrl"
  - "-187_ctrl"
  - "-186_ctrl"
  - "-185_ctrl"
  - "-184_ctrl"
  - "-183_ctrl"
  - "-182_ctrl"
  - "-181_ctrl"
  - "-180_ctrl"
  - "-179_ctrl"
  - "-178_ctrl"
  - "-177_ctrl"
  - "-176_ctrl"
  - "-175_ctrl"
  - "-174_ctrl"
  - "-173_ctrl"
  - "-172_ctrl"
  - "-171_ctrl"
  - "-170_ctrl"
  - "-169_ctrl"
  - "-168_ctrl"
  - "-167_ctrl"
  - "-166_ctrl"
  - "-165_ctrl"
  - "-164_ctrl"
  - "-163_ctrl"
  - "-162_ctrl"
  - "-161_ctrl"
  - "-160_ctrl"
  - "-159_ctrl"
  - "-158_ctrl"
  - "-157_ctrl"
  - "-156_ctrl"
  - "-155_ctrl"
  - "-154_ctrl"
  - "-153_ctrl"
  - "-152_ctrl"
  - "-151_ctrl"
  - "-150_ctrl"
  - "-149_ctrl"
  - "-148_ctrl"
  - "-147_ctrl"
  - "-146_ctrl"
  - "-145_ctrl"
  - "-144_ctrl"
  - "-143_ctrl"
  - "-142_ctrl"
  - "-141_ctrl"
  - "-140_ctrl"
  - "-139_ctrl"
  - "-138_ctrl"
  - "-137_ctrl"
  - "-136_ctrl"
  - "-135_ctrl"
  - "-134_ctrl"
  - "-133_ctrl"
  - "-132_ctrl"
  - "-131_ctrl"
  - "-130_ctrl"
  - "-129_ctrl"
  - "-128_ctrl"
  - "-127_ctrl"
  - "-126_ctrl"
  - "-125_ctrl"
  - "-124_ctrl"
  - "-123_ctrl"
  - "-122_ctrl"
  - "-121_ctrl"
  - "-120_ctrl"
  - "-119_ctrl"
  - "-118_ctrl"
  - "-117_ctrl"
  - "-116_ctrl"
  - "-115_ctrl"
  - "-114_ctrl"
  - "-113_ctrl"
  - "-112_ctrl"
  - "-111_ctrl"
  - "-110_ctrl"
  - "-109_ctrl"
  - "-108_ctrl"
  - "-107_ctrl"
  - "-106_ctrl"
  - "-105_ctrl"
  - "-104_ctrl"
  - "-103_ctrl"
  - "-102_ctrl"
  - "-101_ctrl"
  - "-100_ctrl"
  - "-99_ctrl"
  - "-98_ctrl"
  - "-97_ctrl"
  - "-96_ctrl"
  - "-95_ctrl"
  - "-94_ctrl"
  - "-93_ctrl"
  - "-92_ctrl"
  - "-91_ctrl"
  - "-90_ctrl"
  - "-89_ctrl"
  - "-88_ctrl"
  - "-87_ctrl"
  - "-86_ctrl"
  - "-85_ctrl"
  - "-84_ctrl"
  - "-83_ctrl"
  - "-82_ctrl"
  - "-81_ctrl"
  - "-80_ctrl"
  - "-79_ctrl"
  - "-78_ctrl"
  - "-77_ctrl"
  - "-76_ctrl"
  - "-75_ctrl"
  - "-74_ctrl"
  - "-73_ctrl"
  - "-72_ctrl"
  - "-71_ctrl"
  - "-70_ctrl"
  - "-69_ctrl"
  - "-68_ctrl"
  - "-67_ctrl"
  - "-66_ctrl"
  - "-65_ctrl"
  - "-64_ctrl"
  - "-63_ctrl"
  - "-62_ctrl"
  - "-61_ctrl"
  - "-60_ctrl"
  - "-59_ctrl"
  - "-58_ctrl"
  - "-57_ctrl"
  - "-56_ctrl"
  - "-55_ctrl"
  - "-54_ctrl"
  - "-53_ctrl"
  - "-52_ctrl"
  - "-51_ctrl"
  - "-50_ctrl"
  - "-49_ctrl"
  - "-48_ctrl"
  - "-47_ctrl"
  - "-46_ctrl"
  - "-45_ctrl"
  - "-44_ctrl"
  - "-43_ctrl"
  - "-42_ctrl"
  - "-41_ctrl"
  - "-40_ctrl"
  - "-39_ctrl"
  - "-38_ctrl"
  - "-37_ctrl"
  - "-36_ctrl"
  - "-35_ctrl"
  - "-34_ctrl"
  - "-33_ctrl"
  - "-32_ctrl"
  - "-31_ctrl"
  - "-30_ctrl"
  - "-29_ctrl"
  - "-28_ctrl"
  - "-27_ctrl"
  - "-26_ctrl"
  - "-25_ctrl"
  - "-24_ctrl"
  - "-23_ctrl"
  - "-22_ctrl"
  - "-21_ctrl"
  - "-20_ctrl"
  - "-19_ctrl"
  - "-18_ctrl"
  - "-17_ctrl"
  - "-16_ctrl"
  - "-15_ctrl"
  - "-14_ctrl"
  - "-13_ctrl"
  - "-12_ctrl"
  - "-11_ctrl"
  - "-10_ctrl"
  - "-9_ctrl"
  - "-8_ctrl"
  - "-7_ctrl"
  - "-6_ctrl"
  - "-5_ctrl"
  - "-4_ctrl"
  - "-3_ctrl"
  - "-2_ctrl"
  - "-1_ctrl"
  - "0_ctrl"
  - "1_ctrl"
  - "2_ctrl"
  - "3_ctrl"
  - "4_ctrl"
  - "5_ctrl"
  - "6_ctrl"
  - "7_ctrl"
  - "8_ctrl"
  - "9_ctrl"
  - "10_ctrl"
  - "11_ctrl"
  - "12_ctrl"
  - "13_ctrl"
  - "14_ctrl"
  - "15_ctrl"
  - "16_ctrl"
  - "17_ctrl"
  - "18_ctrl"
  - "19_ctrl"
  - "20_ctrl"
  - "21_ctrl"
  - "22_ctrl"
  - "23_ctrl"
  - "24_ctrl"
  - "25_ctrl"
  - "26_ctrl"
  - "27_ctrl"
  - "28_ctrl"
  - "29_ctrl"
  - "30_ctrl"
  - "31_ctrl"
  - "32_ctrl"
  - "33_ctrl"
  - "34_ctrl"
  - "35_ctrl"
  - "36_ctrl"
  - "37_ctrl"
  - "38_ctrl"
  - "39_ctrl"
  - "40_ctrl"
  - "41_ctrl"
  - "42_ctrl"
  - "43_ctrl"
  - "44_ctrl"
  - "45_ctrl"
  - "46_ctrl"
  - "47_ctrl"
  - "48_ctrl"
  - "49_ctrl"
  - "50_ctrl"
  - "51_ctrl"
  - "52_ctrl"
  - "53_ctrl"
  - "54_ctrl"
  - "55_ctrl"
  - "56_ctrl"
  - "57_ctrl"
  - "58_ctrl"
  - "59_ctrl"
  - "60_ctrl"
  - "61_ctrl"
  - "62_ctrl"
  - "63_ctrl"
  - "64_ctrl"
  - "65_ctrl"
  - "66_ctrl"
  - "67_ctrl"
  - "68_ctrl"
  - "69_ctrl"
  - "70_ctrl"
  - "71_ctrl"
  - "72_ctrl"
  - "73_ctrl"
  - "74_ctrl"
  - "75_ctrl"
  - "76_ctrl"
  - "77_ctrl"
  - "78_ctrl"
  - "79_ctrl"
  - "80_ctrl"
  - "81_ctrl"
  - "82_ctrl"
  - "83_ctrl"
  - "84_ctrl"
  - "85_ctrl"
  - "86_ctrl"
  - "87_ctrl"
  - "88_ctrl"
  - "89_ctrl"
  - "90_ctrl"
  - "91_ctrl"
  - "92_ctrl"
  - "93_ctrl"
  - "94_ctrl"
  - "95_ctrl"
  - "96_ctrl"
  - "97_ctrl"
  - "98_ctrl"
  - "99_ctrl"
  - "100_ctrl"
  - "101_ctrl"
  - "102_ctrl"
  - "103_ctrl"
  - "104_ctrl"
  - "105_ctrl"
  - "106_ctrl"
  - "107_ctrl"
  - "108_ctrl"
  - "109_ctrl"
  - "110_ctrl"
  - "111_ctrl"
  - "112_ctrl"
  - "113_ctrl"
  - "114_ctrl"
  - "115_ctrl"
  - "116_ctrl"
  - "117_ctrl"
  - "118_ctrl"
  - "119_ctrl"
  - "120_ctrl"
  - "121_ctrl"
  - "122_ctrl"
  - "123_ctrl"
  - "124_ctrl"
  - "125_ctrl"
  - "126_ctrl"
  - "127_ctrl"
  - "128_ctrl"
  - "129_ctrl"
  - "130_ctrl"
  - "131_ctrl"
  - "132_ctrl"
  - "133_ctrl"
  - "134_ctrl"
  - "135_ctrl"
  - "136_ctrl"
  - "137_ctrl"
  - "138_ctrl"
  - "139_ctrl"
  - "140_ctrl"
  - "141_ctrl"
  - "142_ctrl"
  - "143_ctrl"
  - "144_ctrl"
  - "145_ctrl"
  - "146_ctrl"
  - "147_ctrl"
  - "148_ctrl"
  - "149_ctrl"
  - "150_ctrl"
  - "151_ctrl"
  - "152_ctrl"
  - "153_ctrl"
  - "154_ctrl"
  - "155_ctrl"
  - "156_ctrl"
  - "157_ctrl"
  - "158_ctrl"
  - "159_ctrl"
  - "160_ctrl"
  - "161_ctrl"
  - "162_ctrl"
  - "163_ctrl"
  - "164_ctrl"
  - "165_ctrl"
  - "166_ctrl"
  - "167_ctrl"
  - "168_ctrl"
  - "169_ctrl"
  - "170_ctrl"
  - "171_ctrl"
  - "172_ctrl"
  - "173_ctrl"
  - "174_ctrl"
  - "175_ctrl"
  - "176_ctrl"
  - "177_ctrl"
  - "178_ctrl"
  - "179_ctrl"
  - "180_ctrl"
  - "181_ctrl"
  - "182_ctrl"
  - "183_ctrl"
  - "184_ctrl"
  - "185_ctrl"
  - "186_ctrl"
  - "187_ctrl"
  - "188_ctrl"
  - "189_ctrl"
  - "190_ctrl"
  - "191_ctrl"
  - "192_ctrl"
  - "193_ctrl"
  - "194_ctrl"
  - "195_ctrl"
  - "196_ctrl"
  - "197_ctrl"
  - "198_ctrl"
  - "199_ctrl"
  - "200_ctrl"
  """
}

test_microc_rotated_yaml_contents = {"globals.yaml":"""
#output_folder: ./runs/gene_expression_microc_rotated_pure_conv/  #gene_expression_prediction_no_H3K27ac_uncoupled/     #gene_expression_cnn_1kbp_401_bins_2_cond_reloaded/ 
#manual_valid_ids_file: ./annotations/manual_validation_ids_chr17.txt  #manual_validation_ids_chr17_uncoupled.txt
checkpoint_interval: 30300 
sample_interval: 30300 
n_epochs: 120
batch_size: 64
optimizer: "adamw"
lr: 0.0001 
device: "cuda"
compute_attributions: true
# latent_sampling: 
#   layers_to_sample:
#     - "input_modules.contact_maps.feature_extractor.conv.0.conv_1"
#     - "input_modules.contact_maps.feature_extractor.conv.1.conv_2"
#     - "input_modules.contact_maps.feature_extractor.conv.1.conv_2"
""",
"input_cnn.yaml": """ 
input_info:
  input_source: ./inputs/landscape_arrays/test/ 
  input_name: gene_expression
  input_type: array

model_config:
  model_type: cnn
  #pre_normalization: "instancenorm"
  model_init_config:
    num_output_features: 512 # before fc_repr_dim
    layers: [4,4]
    kernel_height: 1
    down_stride_width: 2
    first_stride_expansion_width: 1
    first_kernel_expansion_height: 4 #5
    kernel_width: 10  #10
    dilation_factor_width: 2
    dilation_factor_height: 1
    channel_exp_base: 5  #3 #-1
    first_channel_expansion: 2
    rb_do: .3
    stochastic_depth_p: .1
    attention_inclusion_cutoff: 1 #50 #256

""",
"""input_cnn_microc_rotated.yaml""": """ 
input_info:
  input_source: ./inputs/microC_rotated/test/ #arrays_train_100bp_no_H3K27ac_uncoupled/
  input_name: contact_maps
  input_type: array

model_config:
  model_type: cnn
  #pre_normalization: "instancenorm"
  model_init_config:
    num_output_features: 512 #before fc_repr_dim
    layers: [1,2]
    kernel_height: 1
    down_stride_width: 2
    first_stride_expansion_width: 1
    first_kernel_expansion_height: 129 #Height of the array
    kernel_width: 10  #10
    dilation_factor_width: 2
    channel_exp_base: 4  #3 #-1
    first_channel_expansion: 2
    rb_do: .3
    stochastic_depth_p: .1
    attention_inclusion_cutoff: 1 # 50 #128 #256
""",
"fusion.yaml": """
model_config:
  fc_do: 0.4
  fc_task_dim: 256
  layers:
  - 2
  rb_do: 0.4
  stochastic_depth_p: 0.5
model_type: "mlp-residual"
""",
"outputs_2_cond.yaml":"""
output_info:
  output_name: expression_output
  output_source: ./targets/test_targets.csv 
  output_type: tabular

model_config: # <- new
  model_type: linear # <- new

output_type_info:
  con_loss_name: "SmoothL1Loss"
  target_con_columns:
  - "-200_ctrl"
  - "-199_ctrl"
  - "-198_ctrl"
  - "-197_ctrl"
  - "-196_ctrl"
  - "-195_ctrl"
  - "-194_ctrl"
  - "-193_ctrl"
  - "-192_ctrl"
  - "-191_ctrl"
  - "-190_ctrl"
  - "-189_ctrl"
  - "-188_ctrl"
  - "-187_ctrl"
  - "-186_ctrl"
  - "-185_ctrl"
  - "-184_ctrl"
  - "-183_ctrl"
  - "-182_ctrl"
  - "-181_ctrl"
  - "-180_ctrl"
  - "-179_ctrl"
  - "-178_ctrl"
  - "-177_ctrl"
  - "-176_ctrl"
  - "-175_ctrl"
  - "-174_ctrl"
  - "-173_ctrl"
  - "-172_ctrl"
  - "-171_ctrl"
  - "-170_ctrl"
  - "-169_ctrl"
  - "-168_ctrl"
  - "-167_ctrl"
  - "-166_ctrl"
  - "-165_ctrl"
  - "-164_ctrl"
  - "-163_ctrl"
  - "-162_ctrl"
  - "-161_ctrl"
  - "-160_ctrl"
  - "-159_ctrl"
  - "-158_ctrl"
  - "-157_ctrl"
  - "-156_ctrl"
  - "-155_ctrl"
  - "-154_ctrl"
  - "-153_ctrl"
  - "-152_ctrl"
  - "-151_ctrl"
  - "-150_ctrl"
  - "-149_ctrl"
  - "-148_ctrl"
  - "-147_ctrl"
  - "-146_ctrl"
  - "-145_ctrl"
  - "-144_ctrl"
  - "-143_ctrl"
  - "-142_ctrl"
  - "-141_ctrl"
  - "-140_ctrl"
  - "-139_ctrl"
  - "-138_ctrl"
  - "-137_ctrl"
  - "-136_ctrl"
  - "-135_ctrl"
  - "-134_ctrl"
  - "-133_ctrl"
  - "-132_ctrl"
  - "-131_ctrl"
  - "-130_ctrl"
  - "-129_ctrl"
  - "-128_ctrl"
  - "-127_ctrl"
  - "-126_ctrl"
  - "-125_ctrl"
  - "-124_ctrl"
  - "-123_ctrl"
  - "-122_ctrl"
  - "-121_ctrl"
  - "-120_ctrl"
  - "-119_ctrl"
  - "-118_ctrl"
  - "-117_ctrl"
  - "-116_ctrl"
  - "-115_ctrl"
  - "-114_ctrl"
  - "-113_ctrl"
  - "-112_ctrl"
  - "-111_ctrl"
  - "-110_ctrl"
  - "-109_ctrl"
  - "-108_ctrl"
  - "-107_ctrl"
  - "-106_ctrl"
  - "-105_ctrl"
  - "-104_ctrl"
  - "-103_ctrl"
  - "-102_ctrl"
  - "-101_ctrl"
  - "-100_ctrl"
  - "-99_ctrl"
  - "-98_ctrl"
  - "-97_ctrl"
  - "-96_ctrl"
  - "-95_ctrl"
  - "-94_ctrl"
  - "-93_ctrl"
  - "-92_ctrl"
  - "-91_ctrl"
  - "-90_ctrl"
  - "-89_ctrl"
  - "-88_ctrl"
  - "-87_ctrl"
  - "-86_ctrl"
  - "-85_ctrl"
  - "-84_ctrl"
  - "-83_ctrl"
  - "-82_ctrl"
  - "-81_ctrl"
  - "-80_ctrl"
  - "-79_ctrl"
  - "-78_ctrl"
  - "-77_ctrl"
  - "-76_ctrl"
  - "-75_ctrl"
  - "-74_ctrl"
  - "-73_ctrl"
  - "-72_ctrl"
  - "-71_ctrl"
  - "-70_ctrl"
  - "-69_ctrl"
  - "-68_ctrl"
  - "-67_ctrl"
  - "-66_ctrl"
  - "-65_ctrl"
  - "-64_ctrl"
  - "-63_ctrl"
  - "-62_ctrl"
  - "-61_ctrl"
  - "-60_ctrl"
  - "-59_ctrl"
  - "-58_ctrl"
  - "-57_ctrl"
  - "-56_ctrl"
  - "-55_ctrl"
  - "-54_ctrl"
  - "-53_ctrl"
  - "-52_ctrl"
  - "-51_ctrl"
  - "-50_ctrl"
  - "-49_ctrl"
  - "-48_ctrl"
  - "-47_ctrl"
  - "-46_ctrl"
  - "-45_ctrl"
  - "-44_ctrl"
  - "-43_ctrl"
  - "-42_ctrl"
  - "-41_ctrl"
  - "-40_ctrl"
  - "-39_ctrl"
  - "-38_ctrl"
  - "-37_ctrl"
  - "-36_ctrl"
  - "-35_ctrl"
  - "-34_ctrl"
  - "-33_ctrl"
  - "-32_ctrl"
  - "-31_ctrl"
  - "-30_ctrl"
  - "-29_ctrl"
  - "-28_ctrl"
  - "-27_ctrl"
  - "-26_ctrl"
  - "-25_ctrl"
  - "-24_ctrl"
  - "-23_ctrl"
  - "-22_ctrl"
  - "-21_ctrl"
  - "-20_ctrl"
  - "-19_ctrl"
  - "-18_ctrl"
  - "-17_ctrl"
  - "-16_ctrl"
  - "-15_ctrl"
  - "-14_ctrl"
  - "-13_ctrl"
  - "-12_ctrl"
  - "-11_ctrl"
  - "-10_ctrl"
  - "-9_ctrl"
  - "-8_ctrl"
  - "-7_ctrl"
  - "-6_ctrl"
  - "-5_ctrl"
  - "-4_ctrl"
  - "-3_ctrl"
  - "-2_ctrl"
  - "-1_ctrl"
  - "0_ctrl"
  - "1_ctrl"
  - "2_ctrl"
  - "3_ctrl"
  - "4_ctrl"
  - "5_ctrl"
  - "6_ctrl"
  - "7_ctrl"
  - "8_ctrl"
  - "9_ctrl"
  - "10_ctrl"
  - "11_ctrl"
  - "12_ctrl"
  - "13_ctrl"
  - "14_ctrl"
  - "15_ctrl"
  - "16_ctrl"
  - "17_ctrl"
  - "18_ctrl"
  - "19_ctrl"
  - "20_ctrl"
  - "21_ctrl"
  - "22_ctrl"
  - "23_ctrl"
  - "24_ctrl"
  - "25_ctrl"
  - "26_ctrl"
  - "27_ctrl"
  - "28_ctrl"
  - "29_ctrl"
  - "30_ctrl"
  - "31_ctrl"
  - "32_ctrl"
  - "33_ctrl"
  - "34_ctrl"
  - "35_ctrl"
  - "36_ctrl"
  - "37_ctrl"
  - "38_ctrl"
  - "39_ctrl"
  - "40_ctrl"
  - "41_ctrl"
  - "42_ctrl"
  - "43_ctrl"
  - "44_ctrl"
  - "45_ctrl"
  - "46_ctrl"
  - "47_ctrl"
  - "48_ctrl"
  - "49_ctrl"
  - "50_ctrl"
  - "51_ctrl"
  - "52_ctrl"
  - "53_ctrl"
  - "54_ctrl"
  - "55_ctrl"
  - "56_ctrl"
  - "57_ctrl"
  - "58_ctrl"
  - "59_ctrl"
  - "60_ctrl"
  - "61_ctrl"
  - "62_ctrl"
  - "63_ctrl"
  - "64_ctrl"
  - "65_ctrl"
  - "66_ctrl"
  - "67_ctrl"
  - "68_ctrl"
  - "69_ctrl"
  - "70_ctrl"
  - "71_ctrl"
  - "72_ctrl"
  - "73_ctrl"
  - "74_ctrl"
  - "75_ctrl"
  - "76_ctrl"
  - "77_ctrl"
  - "78_ctrl"
  - "79_ctrl"
  - "80_ctrl"
  - "81_ctrl"
  - "82_ctrl"
  - "83_ctrl"
  - "84_ctrl"
  - "85_ctrl"
  - "86_ctrl"
  - "87_ctrl"
  - "88_ctrl"
  - "89_ctrl"
  - "90_ctrl"
  - "91_ctrl"
  - "92_ctrl"
  - "93_ctrl"
  - "94_ctrl"
  - "95_ctrl"
  - "96_ctrl"
  - "97_ctrl"
  - "98_ctrl"
  - "99_ctrl"
  - "100_ctrl"
  - "101_ctrl"
  - "102_ctrl"
  - "103_ctrl"
  - "104_ctrl"
  - "105_ctrl"
  - "106_ctrl"
  - "107_ctrl"
  - "108_ctrl"
  - "109_ctrl"
  - "110_ctrl"
  - "111_ctrl"
  - "112_ctrl"
  - "113_ctrl"
  - "114_ctrl"
  - "115_ctrl"
  - "116_ctrl"
  - "117_ctrl"
  - "118_ctrl"
  - "119_ctrl"
  - "120_ctrl"
  - "121_ctrl"
  - "122_ctrl"
  - "123_ctrl"
  - "124_ctrl"
  - "125_ctrl"
  - "126_ctrl"
  - "127_ctrl"
  - "128_ctrl"
  - "129_ctrl"
  - "130_ctrl"
  - "131_ctrl"
  - "132_ctrl"
  - "133_ctrl"
  - "134_ctrl"
  - "135_ctrl"
  - "136_ctrl"
  - "137_ctrl"
  - "138_ctrl"
  - "139_ctrl"
  - "140_ctrl"
  - "141_ctrl"
  - "142_ctrl"
  - "143_ctrl"
  - "144_ctrl"
  - "145_ctrl"
  - "146_ctrl"
  - "147_ctrl"
  - "148_ctrl"
  - "149_ctrl"
  - "150_ctrl"
  - "151_ctrl"
  - "152_ctrl"
  - "153_ctrl"
  - "154_ctrl"
  - "155_ctrl"
  - "156_ctrl"
  - "157_ctrl"
  - "158_ctrl"
  - "159_ctrl"
  - "160_ctrl"
  - "161_ctrl"
  - "162_ctrl"
  - "163_ctrl"
  - "164_ctrl"
  - "165_ctrl"
  - "166_ctrl"
  - "167_ctrl"
  - "168_ctrl"
  - "169_ctrl"
  - "170_ctrl"
  - "171_ctrl"
  - "172_ctrl"
  - "173_ctrl"
  - "174_ctrl"
  - "175_ctrl"
  - "176_ctrl"
  - "177_ctrl"
  - "178_ctrl"
  - "179_ctrl"
  - "180_ctrl"
  - "181_ctrl"
  - "182_ctrl"
  - "183_ctrl"
  - "184_ctrl"
  - "185_ctrl"
  - "186_ctrl"
  - "187_ctrl"
  - "188_ctrl"
  - "189_ctrl"
  - "190_ctrl"
  - "191_ctrl"
  - "192_ctrl"
  - "193_ctrl"
  - "194_ctrl"
  - "195_ctrl"
  - "196_ctrl"
  - "197_ctrl"
  - "198_ctrl"
  - "199_ctrl"
  - "200_ctrl"
  """
}

training_attention_yaml_contents = {"globals.yaml":"""
output_folder: ./runs/gene_expression_only_chrom_attention/  #gene_expression_prediction_no_H3K27ac_uncoupled/     #gene_expression_cnn_1kbp_401_bins_2_cond_reloaded/ 
manual_valid_ids_file: ./annotations/manual_validation_ids_chr17.txt  #manual_validation_ids_chr17_uncoupled.txt
checkpoint_interval: 30300 # 100 epochs #60000
sample_interval: 30300 #60000
n_epochs: 120
batch_size: 64
optimizer: "adamw"
lr: 0.0001 #0.0001
device: "cuda"
compute_attributions: false
# latent_sampling: 
#   layers_to_sample:
#     - "input_modules.contact_maps.feature_extractor.conv.0.conv_1"
#     - "input_modules.contact_maps.feature_extractor.conv.1.conv_2"
#     - "input_modules.contact_maps.feature_extractor.conv.2.conv_2"
#     - "input_modules.contact_maps.feature_extractor.conv.3.conv_2"
# attribution_background_samples: 512
# attributions_every_sample_factor: 1
#pretrained_checkpoint: best_models/gene_expression_exformer_unlimited_chrom_and_micro_with_attention_model_117600_perf-average=0.8435.pt 

""",
"input_cnn.yaml": """ 
input_info:
  input_source: ./inputs/landscape_arrays/training/ 
  input_name: gene_expression
  input_type: array

model_config:
  model_type: cnn
  #pre_normalization: "instancenorm"
  model_init_config:
    num_output_features: 512 # before fc_repr_dim
    layers: [4,4]
    kernel_height: 1
    down_stride_width: 2
    first_stride_expansion_width: 1
    first_kernel_expansion_height: 4 #5
    kernel_width: 10  #10
    dilation_factor_width: 2
    dilation_factor_height: 1
    channel_exp_base: 5  #3 #-1
    first_channel_expansion: 2
    rb_do: .3
    stochastic_depth_p: .1
    attention_inclusion_cutoff: 50 #256

""",
"fusion.yaml": """
model_config:
  fc_do: 0.4
  fc_task_dim: 256
  layers:
  - 2
  rb_do: 0.4
  stochastic_depth_p: 0.5
model_type: "mlp-residual"
""",
"outputs_2_cond.yaml":"""
output_info:
  output_name: expression_output
  output_source: ./targets/training_targets.csv 
  output_type: tabular

model_config: # <- new
  model_type: linear # <- new

output_type_info:
  con_loss_name: "SmoothL1Loss"
  target_con_columns:
  - "-200_ctrl"
  - "-199_ctrl"
  - "-198_ctrl"
  - "-197_ctrl"
  - "-196_ctrl"
  - "-195_ctrl"
  - "-194_ctrl"
  - "-193_ctrl"
  - "-192_ctrl"
  - "-191_ctrl"
  - "-190_ctrl"
  - "-189_ctrl"
  - "-188_ctrl"
  - "-187_ctrl"
  - "-186_ctrl"
  - "-185_ctrl"
  - "-184_ctrl"
  - "-183_ctrl"
  - "-182_ctrl"
  - "-181_ctrl"
  - "-180_ctrl"
  - "-179_ctrl"
  - "-178_ctrl"
  - "-177_ctrl"
  - "-176_ctrl"
  - "-175_ctrl"
  - "-174_ctrl"
  - "-173_ctrl"
  - "-172_ctrl"
  - "-171_ctrl"
  - "-170_ctrl"
  - "-169_ctrl"
  - "-168_ctrl"
  - "-167_ctrl"
  - "-166_ctrl"
  - "-165_ctrl"
  - "-164_ctrl"
  - "-163_ctrl"
  - "-162_ctrl"
  - "-161_ctrl"
  - "-160_ctrl"
  - "-159_ctrl"
  - "-158_ctrl"
  - "-157_ctrl"
  - "-156_ctrl"
  - "-155_ctrl"
  - "-154_ctrl"
  - "-153_ctrl"
  - "-152_ctrl"
  - "-151_ctrl"
  - "-150_ctrl"
  - "-149_ctrl"
  - "-148_ctrl"
  - "-147_ctrl"
  - "-146_ctrl"
  - "-145_ctrl"
  - "-144_ctrl"
  - "-143_ctrl"
  - "-142_ctrl"
  - "-141_ctrl"
  - "-140_ctrl"
  - "-139_ctrl"
  - "-138_ctrl"
  - "-137_ctrl"
  - "-136_ctrl"
  - "-135_ctrl"
  - "-134_ctrl"
  - "-133_ctrl"
  - "-132_ctrl"
  - "-131_ctrl"
  - "-130_ctrl"
  - "-129_ctrl"
  - "-128_ctrl"
  - "-127_ctrl"
  - "-126_ctrl"
  - "-125_ctrl"
  - "-124_ctrl"
  - "-123_ctrl"
  - "-122_ctrl"
  - "-121_ctrl"
  - "-120_ctrl"
  - "-119_ctrl"
  - "-118_ctrl"
  - "-117_ctrl"
  - "-116_ctrl"
  - "-115_ctrl"
  - "-114_ctrl"
  - "-113_ctrl"
  - "-112_ctrl"
  - "-111_ctrl"
  - "-110_ctrl"
  - "-109_ctrl"
  - "-108_ctrl"
  - "-107_ctrl"
  - "-106_ctrl"
  - "-105_ctrl"
  - "-104_ctrl"
  - "-103_ctrl"
  - "-102_ctrl"
  - "-101_ctrl"
  - "-100_ctrl"
  - "-99_ctrl"
  - "-98_ctrl"
  - "-97_ctrl"
  - "-96_ctrl"
  - "-95_ctrl"
  - "-94_ctrl"
  - "-93_ctrl"
  - "-92_ctrl"
  - "-91_ctrl"
  - "-90_ctrl"
  - "-89_ctrl"
  - "-88_ctrl"
  - "-87_ctrl"
  - "-86_ctrl"
  - "-85_ctrl"
  - "-84_ctrl"
  - "-83_ctrl"
  - "-82_ctrl"
  - "-81_ctrl"
  - "-80_ctrl"
  - "-79_ctrl"
  - "-78_ctrl"
  - "-77_ctrl"
  - "-76_ctrl"
  - "-75_ctrl"
  - "-74_ctrl"
  - "-73_ctrl"
  - "-72_ctrl"
  - "-71_ctrl"
  - "-70_ctrl"
  - "-69_ctrl"
  - "-68_ctrl"
  - "-67_ctrl"
  - "-66_ctrl"
  - "-65_ctrl"
  - "-64_ctrl"
  - "-63_ctrl"
  - "-62_ctrl"
  - "-61_ctrl"
  - "-60_ctrl"
  - "-59_ctrl"
  - "-58_ctrl"
  - "-57_ctrl"
  - "-56_ctrl"
  - "-55_ctrl"
  - "-54_ctrl"
  - "-53_ctrl"
  - "-52_ctrl"
  - "-51_ctrl"
  - "-50_ctrl"
  - "-49_ctrl"
  - "-48_ctrl"
  - "-47_ctrl"
  - "-46_ctrl"
  - "-45_ctrl"
  - "-44_ctrl"
  - "-43_ctrl"
  - "-42_ctrl"
  - "-41_ctrl"
  - "-40_ctrl"
  - "-39_ctrl"
  - "-38_ctrl"
  - "-37_ctrl"
  - "-36_ctrl"
  - "-35_ctrl"
  - "-34_ctrl"
  - "-33_ctrl"
  - "-32_ctrl"
  - "-31_ctrl"
  - "-30_ctrl"
  - "-29_ctrl"
  - "-28_ctrl"
  - "-27_ctrl"
  - "-26_ctrl"
  - "-25_ctrl"
  - "-24_ctrl"
  - "-23_ctrl"
  - "-22_ctrl"
  - "-21_ctrl"
  - "-20_ctrl"
  - "-19_ctrl"
  - "-18_ctrl"
  - "-17_ctrl"
  - "-16_ctrl"
  - "-15_ctrl"
  - "-14_ctrl"
  - "-13_ctrl"
  - "-12_ctrl"
  - "-11_ctrl"
  - "-10_ctrl"
  - "-9_ctrl"
  - "-8_ctrl"
  - "-7_ctrl"
  - "-6_ctrl"
  - "-5_ctrl"
  - "-4_ctrl"
  - "-3_ctrl"
  - "-2_ctrl"
  - "-1_ctrl"
  - "0_ctrl"
  - "1_ctrl"
  - "2_ctrl"
  - "3_ctrl"
  - "4_ctrl"
  - "5_ctrl"
  - "6_ctrl"
  - "7_ctrl"
  - "8_ctrl"
  - "9_ctrl"
  - "10_ctrl"
  - "11_ctrl"
  - "12_ctrl"
  - "13_ctrl"
  - "14_ctrl"
  - "15_ctrl"
  - "16_ctrl"
  - "17_ctrl"
  - "18_ctrl"
  - "19_ctrl"
  - "20_ctrl"
  - "21_ctrl"
  - "22_ctrl"
  - "23_ctrl"
  - "24_ctrl"
  - "25_ctrl"
  - "26_ctrl"
  - "27_ctrl"
  - "28_ctrl"
  - "29_ctrl"
  - "30_ctrl"
  - "31_ctrl"
  - "32_ctrl"
  - "33_ctrl"
  - "34_ctrl"
  - "35_ctrl"
  - "36_ctrl"
  - "37_ctrl"
  - "38_ctrl"
  - "39_ctrl"
  - "40_ctrl"
  - "41_ctrl"
  - "42_ctrl"
  - "43_ctrl"
  - "44_ctrl"
  - "45_ctrl"
  - "46_ctrl"
  - "47_ctrl"
  - "48_ctrl"
  - "49_ctrl"
  - "50_ctrl"
  - "51_ctrl"
  - "52_ctrl"
  - "53_ctrl"
  - "54_ctrl"
  - "55_ctrl"
  - "56_ctrl"
  - "57_ctrl"
  - "58_ctrl"
  - "59_ctrl"
  - "60_ctrl"
  - "61_ctrl"
  - "62_ctrl"
  - "63_ctrl"
  - "64_ctrl"
  - "65_ctrl"
  - "66_ctrl"
  - "67_ctrl"
  - "68_ctrl"
  - "69_ctrl"
  - "70_ctrl"
  - "71_ctrl"
  - "72_ctrl"
  - "73_ctrl"
  - "74_ctrl"
  - "75_ctrl"
  - "76_ctrl"
  - "77_ctrl"
  - "78_ctrl"
  - "79_ctrl"
  - "80_ctrl"
  - "81_ctrl"
  - "82_ctrl"
  - "83_ctrl"
  - "84_ctrl"
  - "85_ctrl"
  - "86_ctrl"
  - "87_ctrl"
  - "88_ctrl"
  - "89_ctrl"
  - "90_ctrl"
  - "91_ctrl"
  - "92_ctrl"
  - "93_ctrl"
  - "94_ctrl"
  - "95_ctrl"
  - "96_ctrl"
  - "97_ctrl"
  - "98_ctrl"
  - "99_ctrl"
  - "100_ctrl"
  - "101_ctrl"
  - "102_ctrl"
  - "103_ctrl"
  - "104_ctrl"
  - "105_ctrl"
  - "106_ctrl"
  - "107_ctrl"
  - "108_ctrl"
  - "109_ctrl"
  - "110_ctrl"
  - "111_ctrl"
  - "112_ctrl"
  - "113_ctrl"
  - "114_ctrl"
  - "115_ctrl"
  - "116_ctrl"
  - "117_ctrl"
  - "118_ctrl"
  - "119_ctrl"
  - "120_ctrl"
  - "121_ctrl"
  - "122_ctrl"
  - "123_ctrl"
  - "124_ctrl"
  - "125_ctrl"
  - "126_ctrl"
  - "127_ctrl"
  - "128_ctrl"
  - "129_ctrl"
  - "130_ctrl"
  - "131_ctrl"
  - "132_ctrl"
  - "133_ctrl"
  - "134_ctrl"
  - "135_ctrl"
  - "136_ctrl"
  - "137_ctrl"
  - "138_ctrl"
  - "139_ctrl"
  - "140_ctrl"
  - "141_ctrl"
  - "142_ctrl"
  - "143_ctrl"
  - "144_ctrl"
  - "145_ctrl"
  - "146_ctrl"
  - "147_ctrl"
  - "148_ctrl"
  - "149_ctrl"
  - "150_ctrl"
  - "151_ctrl"
  - "152_ctrl"
  - "153_ctrl"
  - "154_ctrl"
  - "155_ctrl"
  - "156_ctrl"
  - "157_ctrl"
  - "158_ctrl"
  - "159_ctrl"
  - "160_ctrl"
  - "161_ctrl"
  - "162_ctrl"
  - "163_ctrl"
  - "164_ctrl"
  - "165_ctrl"
  - "166_ctrl"
  - "167_ctrl"
  - "168_ctrl"
  - "169_ctrl"
  - "170_ctrl"
  - "171_ctrl"
  - "172_ctrl"
  - "173_ctrl"
  - "174_ctrl"
  - "175_ctrl"
  - "176_ctrl"
  - "177_ctrl"
  - "178_ctrl"
  - "179_ctrl"
  - "180_ctrl"
  - "181_ctrl"
  - "182_ctrl"
  - "183_ctrl"
  - "184_ctrl"
  - "185_ctrl"
  - "186_ctrl"
  - "187_ctrl"
  - "188_ctrl"
  - "189_ctrl"
  - "190_ctrl"
  - "191_ctrl"
  - "192_ctrl"
  - "193_ctrl"
  - "194_ctrl"
  - "195_ctrl"
  - "196_ctrl"
  - "197_ctrl"
  - "198_ctrl"
  - "199_ctrl"
  - "200_ctrl"
  """
}

test_attention_yaml_contents = {"fusion.yaml":"""  
model_config:
  fc_do: 0.4
  fc_task_dim: 256
  layers:
  - 2
  rb_do: 0.4
  stochastic_depth_p: 0.5
model_type: "mlp-residual"
""",
"globals.yaml": """

checkpoint_interval: 30300 # 100 epochs #60000
sample_interval: 30300 #60000
n_epochs: 120
batch_size: 64
optimizer: "adamw"
lr: 0.0001 #0.0001
device: "cuda"
compute_attributions: false

""",
"input_cnn.yaml": """
input_info:
  input_source: ./inputs/landscape_arrays/test/  #inserted_enhancer_test #inputs/silenced_arrays/silenced_arrays_H2B_S.D 
  #./data/parsed_data/inputs/arrays_train_100bp_no_H3K27ac/ #arrays_train_100bp_no_H3K27ac_uncoupled/
  input_name: gene_expression
  input_type: array

model_config:
  model_type: cnn
  #pre_normalization: "instancenorm"
  model_init_config:
    num_output_features: 512 # before fc_repr_dim
    layers: [4,4]
    kernel_height: 1
    down_stride_width: 2
    first_stride_expansion_width: 1
    first_kernel_expansion_height: 4 #5
    kernel_width: 10  #10
    dilation_factor_width: 2
    dilation_factor_height: 1
    channel_exp_base: 5  #3 #-1
    first_channel_expansion: 2
    rb_do: .3
    stochastic_depth_p: .1
    attention_inclusion_cutoff: 50 #256
    """,
"outputs_2_cond.yaml": """
output_info:
  output_name: expression_output
  output_source: ./targets/test_targets.csv 
  #./data/parsed_data/targets/target_arrays_perturbational_inserted_enhancer.csv #target_arrays_perturbational_S.D.csv #target_arrays_1kbp_401_bins_2_conditions_decareads_abs.csv
  output_type: tabular

model_config: # <- new
  model_type: linear # <- new

output_type_info:
  con_loss_name: "SmoothL1Loss"
  target_con_columns:
  - "-200_ctrl"
  - "-199_ctrl"
  - "-198_ctrl"
  - "-197_ctrl"
  - "-196_ctrl"
  - "-195_ctrl"
  - "-194_ctrl"
  - "-193_ctrl"
  - "-192_ctrl"
  - "-191_ctrl"
  - "-190_ctrl"
  - "-189_ctrl"
  - "-188_ctrl"
  - "-187_ctrl"
  - "-186_ctrl"
  - "-185_ctrl"
  - "-184_ctrl"
  - "-183_ctrl"
  - "-182_ctrl"
  - "-181_ctrl"
  - "-180_ctrl"
  - "-179_ctrl"
  - "-178_ctrl"
  - "-177_ctrl"
  - "-176_ctrl"
  - "-175_ctrl"
  - "-174_ctrl"
  - "-173_ctrl"
  - "-172_ctrl"
  - "-171_ctrl"
  - "-170_ctrl"
  - "-169_ctrl"
  - "-168_ctrl"
  - "-167_ctrl"
  - "-166_ctrl"
  - "-165_ctrl"
  - "-164_ctrl"
  - "-163_ctrl"
  - "-162_ctrl"
  - "-161_ctrl"
  - "-160_ctrl"
  - "-159_ctrl"
  - "-158_ctrl"
  - "-157_ctrl"
  - "-156_ctrl"
  - "-155_ctrl"
  - "-154_ctrl"
  - "-153_ctrl"
  - "-152_ctrl"
  - "-151_ctrl"
  - "-150_ctrl"
  - "-149_ctrl"
  - "-148_ctrl"
  - "-147_ctrl"
  - "-146_ctrl"
  - "-145_ctrl"
  - "-144_ctrl"
  - "-143_ctrl"
  - "-142_ctrl"
  - "-141_ctrl"
  - "-140_ctrl"
  - "-139_ctrl"
  - "-138_ctrl"
  - "-137_ctrl"
  - "-136_ctrl"
  - "-135_ctrl"
  - "-134_ctrl"
  - "-133_ctrl"
  - "-132_ctrl"
  - "-131_ctrl"
  - "-130_ctrl"
  - "-129_ctrl"
  - "-128_ctrl"
  - "-127_ctrl"
  - "-126_ctrl"
  - "-125_ctrl"
  - "-124_ctrl"
  - "-123_ctrl"
  - "-122_ctrl"
  - "-121_ctrl"
  - "-120_ctrl"
  - "-119_ctrl"
  - "-118_ctrl"
  - "-117_ctrl"
  - "-116_ctrl"
  - "-115_ctrl"
  - "-114_ctrl"
  - "-113_ctrl"
  - "-112_ctrl"
  - "-111_ctrl"
  - "-110_ctrl"
  - "-109_ctrl"
  - "-108_ctrl"
  - "-107_ctrl"
  - "-106_ctrl"
  - "-105_ctrl"
  - "-104_ctrl"
  - "-103_ctrl"
  - "-102_ctrl"
  - "-101_ctrl"
  - "-100_ctrl"
  - "-99_ctrl"
  - "-98_ctrl"
  - "-97_ctrl"
  - "-96_ctrl"
  - "-95_ctrl"
  - "-94_ctrl"
  - "-93_ctrl"
  - "-92_ctrl"
  - "-91_ctrl"
  - "-90_ctrl"
  - "-89_ctrl"
  - "-88_ctrl"
  - "-87_ctrl"
  - "-86_ctrl"
  - "-85_ctrl"
  - "-84_ctrl"
  - "-83_ctrl"
  - "-82_ctrl"
  - "-81_ctrl"
  - "-80_ctrl"
  - "-79_ctrl"
  - "-78_ctrl"
  - "-77_ctrl"
  - "-76_ctrl"
  - "-75_ctrl"
  - "-74_ctrl"
  - "-73_ctrl"
  - "-72_ctrl"
  - "-71_ctrl"
  - "-70_ctrl"
  - "-69_ctrl"
  - "-68_ctrl"
  - "-67_ctrl"
  - "-66_ctrl"
  - "-65_ctrl"
  - "-64_ctrl"
  - "-63_ctrl"
  - "-62_ctrl"
  - "-61_ctrl"
  - "-60_ctrl"
  - "-59_ctrl"
  - "-58_ctrl"
  - "-57_ctrl"
  - "-56_ctrl"
  - "-55_ctrl"
  - "-54_ctrl"
  - "-53_ctrl"
  - "-52_ctrl"
  - "-51_ctrl"
  - "-50_ctrl"
  - "-49_ctrl"
  - "-48_ctrl"
  - "-47_ctrl"
  - "-46_ctrl"
  - "-45_ctrl"
  - "-44_ctrl"
  - "-43_ctrl"
  - "-42_ctrl"
  - "-41_ctrl"
  - "-40_ctrl"
  - "-39_ctrl"
  - "-38_ctrl"
  - "-37_ctrl"
  - "-36_ctrl"
  - "-35_ctrl"
  - "-34_ctrl"
  - "-33_ctrl"
  - "-32_ctrl"
  - "-31_ctrl"
  - "-30_ctrl"
  - "-29_ctrl"
  - "-28_ctrl"
  - "-27_ctrl"
  - "-26_ctrl"
  - "-25_ctrl"
  - "-24_ctrl"
  - "-23_ctrl"
  - "-22_ctrl"
  - "-21_ctrl"
  - "-20_ctrl"
  - "-19_ctrl"
  - "-18_ctrl"
  - "-17_ctrl"
  - "-16_ctrl"
  - "-15_ctrl"
  - "-14_ctrl"
  - "-13_ctrl"
  - "-12_ctrl"
  - "-11_ctrl"
  - "-10_ctrl"
  - "-9_ctrl"
  - "-8_ctrl"
  - "-7_ctrl"
  - "-6_ctrl"
  - "-5_ctrl"
  - "-4_ctrl"
  - "-3_ctrl"
  - "-2_ctrl"
  - "-1_ctrl"
  - "0_ctrl"
  - "1_ctrl"
  - "2_ctrl"
  - "3_ctrl"
  - "4_ctrl"
  - "5_ctrl"
  - "6_ctrl"
  - "7_ctrl"
  - "8_ctrl"
  - "9_ctrl"
  - "10_ctrl"
  - "11_ctrl"
  - "12_ctrl"
  - "13_ctrl"
  - "14_ctrl"
  - "15_ctrl"
  - "16_ctrl"
  - "17_ctrl"
  - "18_ctrl"
  - "19_ctrl"
  - "20_ctrl"
  - "21_ctrl"
  - "22_ctrl"
  - "23_ctrl"
  - "24_ctrl"
  - "25_ctrl"
  - "26_ctrl"
  - "27_ctrl"
  - "28_ctrl"
  - "29_ctrl"
  - "30_ctrl"
  - "31_ctrl"
  - "32_ctrl"
  - "33_ctrl"
  - "34_ctrl"
  - "35_ctrl"
  - "36_ctrl"
  - "37_ctrl"
  - "38_ctrl"
  - "39_ctrl"
  - "40_ctrl"
  - "41_ctrl"
  - "42_ctrl"
  - "43_ctrl"
  - "44_ctrl"
  - "45_ctrl"
  - "46_ctrl"
  - "47_ctrl"
  - "48_ctrl"
  - "49_ctrl"
  - "50_ctrl"
  - "51_ctrl"
  - "52_ctrl"
  - "53_ctrl"
  - "54_ctrl"
  - "55_ctrl"
  - "56_ctrl"
  - "57_ctrl"
  - "58_ctrl"
  - "59_ctrl"
  - "60_ctrl"
  - "61_ctrl"
  - "62_ctrl"
  - "63_ctrl"
  - "64_ctrl"
  - "65_ctrl"
  - "66_ctrl"
  - "67_ctrl"
  - "68_ctrl"
  - "69_ctrl"
  - "70_ctrl"
  - "71_ctrl"
  - "72_ctrl"
  - "73_ctrl"
  - "74_ctrl"
  - "75_ctrl"
  - "76_ctrl"
  - "77_ctrl"
  - "78_ctrl"
  - "79_ctrl"
  - "80_ctrl"
  - "81_ctrl"
  - "82_ctrl"
  - "83_ctrl"
  - "84_ctrl"
  - "85_ctrl"
  - "86_ctrl"
  - "87_ctrl"
  - "88_ctrl"
  - "89_ctrl"
  - "90_ctrl"
  - "91_ctrl"
  - "92_ctrl"
  - "93_ctrl"
  - "94_ctrl"
  - "95_ctrl"
  - "96_ctrl"
  - "97_ctrl"
  - "98_ctrl"
  - "99_ctrl"
  - "100_ctrl"
  - "101_ctrl"
  - "102_ctrl"
  - "103_ctrl"
  - "104_ctrl"
  - "105_ctrl"
  - "106_ctrl"
  - "107_ctrl"
  - "108_ctrl"
  - "109_ctrl"
  - "110_ctrl"
  - "111_ctrl"
  - "112_ctrl"
  - "113_ctrl"
  - "114_ctrl"
  - "115_ctrl"
  - "116_ctrl"
  - "117_ctrl"
  - "118_ctrl"
  - "119_ctrl"
  - "120_ctrl"
  - "121_ctrl"
  - "122_ctrl"
  - "123_ctrl"
  - "124_ctrl"
  - "125_ctrl"
  - "126_ctrl"
  - "127_ctrl"
  - "128_ctrl"
  - "129_ctrl"
  - "130_ctrl"
  - "131_ctrl"
  - "132_ctrl"
  - "133_ctrl"
  - "134_ctrl"
  - "135_ctrl"
  - "136_ctrl"
  - "137_ctrl"
  - "138_ctrl"
  - "139_ctrl"
  - "140_ctrl"
  - "141_ctrl"
  - "142_ctrl"
  - "143_ctrl"
  - "144_ctrl"
  - "145_ctrl"
  - "146_ctrl"
  - "147_ctrl"
  - "148_ctrl"
  - "149_ctrl"
  - "150_ctrl"
  - "151_ctrl"
  - "152_ctrl"
  - "153_ctrl"
  - "154_ctrl"
  - "155_ctrl"
  - "156_ctrl"
  - "157_ctrl"
  - "158_ctrl"
  - "159_ctrl"
  - "160_ctrl"
  - "161_ctrl"
  - "162_ctrl"
  - "163_ctrl"
  - "164_ctrl"
  - "165_ctrl"
  - "166_ctrl"
  - "167_ctrl"
  - "168_ctrl"
  - "169_ctrl"
  - "170_ctrl"
  - "171_ctrl"
  - "172_ctrl"
  - "173_ctrl"
  - "174_ctrl"
  - "175_ctrl"
  - "176_ctrl"
  - "177_ctrl"
  - "178_ctrl"
  - "179_ctrl"
  - "180_ctrl"
  - "181_ctrl"
  - "182_ctrl"
  - "183_ctrl"
  - "184_ctrl"
  - "185_ctrl"
  - "186_ctrl"
  - "187_ctrl"
  - "188_ctrl"
  - "189_ctrl"
  - "190_ctrl"
  - "191_ctrl"
  - "192_ctrl"
  - "193_ctrl"
  - "194_ctrl"
  - "195_ctrl"
  - "196_ctrl"
  - "197_ctrl"
  - "198_ctrl"
  - "199_ctrl"
  - "200_ctrl"
  """ }

training_microc_latents_yaml_contents = {"globals.yaml":"""
output_folder: ./runs/gene_expression_microc_pure_conv_latents/  #gene_expression_prediction_no_H3K27ac_uncoupled/     #gene_expression_cnn_1kbp_401_bins_2_cond_reloaded/ 
#manual_valid_ids_file: ./annotations/manual_validation_ids_chr17.txt  #manual_validation_ids_chr17_uncoupled.txt
checkpoint_interval: 1200
sample_interval: 1200 
n_epochs: 120
batch_size: 64
optimizer: "adamw"
lr: 0.0001 
device: "cuda"
compute_attributions: false
latent_sampling: 
  layers_to_sample:
    - "input_modules.contact_maps.feature_extractor.conv.0.conv_1"
    - "input_modules.contact_maps.feature_extractor.conv.1.conv_1"
    - "input_modules.contact_maps.feature_extractor.conv.2.conv_1"
    - "input_modules.contact_maps.feature_extractor.conv.3.conv_1"
""",
"input_cnn.yaml": """ 
input_info:
  input_source: ./inputs/landscape_arrays/test/ 
  input_name: gene_expression
  input_type: array

model_config:
  model_type: cnn
  #pre_normalization: "instancenorm"
  model_init_config:
    num_output_features: 512 # before fc_repr_dim
    layers: [4,4]
    kernel_height: 1
    down_stride_width: 2
    first_stride_expansion_width: 1
    first_kernel_expansion_height: 4 #5
    kernel_width: 10  #10
    dilation_factor_width: 2
    dilation_factor_height: 1
    channel_exp_base: 5  #3 #-1
    first_channel_expansion: 2
    rb_do: .3
    stochastic_depth_p: .1
    attention_inclusion_cutoff: 1 #50 #256

""",
"""input_cnn_microc.yaml""": """
input_info:
  input_source: ./inputs/microC/test/ 
  input_name: contact_maps
  input_type: array

model_config:
  model_type: cnn
  #pre_normalization: "instancenorm"
  model_init_config:
    num_output_features: 512 #before fc_repr_dim
    layers: [1,2]
    kernel_height: 5
    down_stride_width: 2
    down_stride_height: 2 #5
    kernel_width: 5  #10
    dilation_factor_width: 2
    dilation_factor_height: 2
    channel_exp_base: 2  #3 #-1
    first_channel_expansion: 2
    rb_do: .3
    stochastic_depth_p: .1
    attention_inclusion_cutoff: 1 #128 #256
""",
"fusion.yaml": """
model_config:
  fc_do: 0.4
  fc_task_dim: 256
  layers:
  - 2
  rb_do: 0.4
  stochastic_depth_p: 0.5
model_type: "mlp-residual"
""",
"outputs_2_cond.yaml":"""
output_info:
  output_name: expression_output
  output_source: ./targets/test_targets.csv 
  output_type: tabular

model_config: # <- new
  model_type: linear # <- new

output_type_info:
  con_loss_name: "SmoothL1Loss"
  target_con_columns:
  - "-200_ctrl"
  - "-199_ctrl"
  - "-198_ctrl"
  - "-197_ctrl"
  - "-196_ctrl"
  - "-195_ctrl"
  - "-194_ctrl"
  - "-193_ctrl"
  - "-192_ctrl"
  - "-191_ctrl"
  - "-190_ctrl"
  - "-189_ctrl"
  - "-188_ctrl"
  - "-187_ctrl"
  - "-186_ctrl"
  - "-185_ctrl"
  - "-184_ctrl"
  - "-183_ctrl"
  - "-182_ctrl"
  - "-181_ctrl"
  - "-180_ctrl"
  - "-179_ctrl"
  - "-178_ctrl"
  - "-177_ctrl"
  - "-176_ctrl"
  - "-175_ctrl"
  - "-174_ctrl"
  - "-173_ctrl"
  - "-172_ctrl"
  - "-171_ctrl"
  - "-170_ctrl"
  - "-169_ctrl"
  - "-168_ctrl"
  - "-167_ctrl"
  - "-166_ctrl"
  - "-165_ctrl"
  - "-164_ctrl"
  - "-163_ctrl"
  - "-162_ctrl"
  - "-161_ctrl"
  - "-160_ctrl"
  - "-159_ctrl"
  - "-158_ctrl"
  - "-157_ctrl"
  - "-156_ctrl"
  - "-155_ctrl"
  - "-154_ctrl"
  - "-153_ctrl"
  - "-152_ctrl"
  - "-151_ctrl"
  - "-150_ctrl"
  - "-149_ctrl"
  - "-148_ctrl"
  - "-147_ctrl"
  - "-146_ctrl"
  - "-145_ctrl"
  - "-144_ctrl"
  - "-143_ctrl"
  - "-142_ctrl"
  - "-141_ctrl"
  - "-140_ctrl"
  - "-139_ctrl"
  - "-138_ctrl"
  - "-137_ctrl"
  - "-136_ctrl"
  - "-135_ctrl"
  - "-134_ctrl"
  - "-133_ctrl"
  - "-132_ctrl"
  - "-131_ctrl"
  - "-130_ctrl"
  - "-129_ctrl"
  - "-128_ctrl"
  - "-127_ctrl"
  - "-126_ctrl"
  - "-125_ctrl"
  - "-124_ctrl"
  - "-123_ctrl"
  - "-122_ctrl"
  - "-121_ctrl"
  - "-120_ctrl"
  - "-119_ctrl"
  - "-118_ctrl"
  - "-117_ctrl"
  - "-116_ctrl"
  - "-115_ctrl"
  - "-114_ctrl"
  - "-113_ctrl"
  - "-112_ctrl"
  - "-111_ctrl"
  - "-110_ctrl"
  - "-109_ctrl"
  - "-108_ctrl"
  - "-107_ctrl"
  - "-106_ctrl"
  - "-105_ctrl"
  - "-104_ctrl"
  - "-103_ctrl"
  - "-102_ctrl"
  - "-101_ctrl"
  - "-100_ctrl"
  - "-99_ctrl"
  - "-98_ctrl"
  - "-97_ctrl"
  - "-96_ctrl"
  - "-95_ctrl"
  - "-94_ctrl"
  - "-93_ctrl"
  - "-92_ctrl"
  - "-91_ctrl"
  - "-90_ctrl"
  - "-89_ctrl"
  - "-88_ctrl"
  - "-87_ctrl"
  - "-86_ctrl"
  - "-85_ctrl"
  - "-84_ctrl"
  - "-83_ctrl"
  - "-82_ctrl"
  - "-81_ctrl"
  - "-80_ctrl"
  - "-79_ctrl"
  - "-78_ctrl"
  - "-77_ctrl"
  - "-76_ctrl"
  - "-75_ctrl"
  - "-74_ctrl"
  - "-73_ctrl"
  - "-72_ctrl"
  - "-71_ctrl"
  - "-70_ctrl"
  - "-69_ctrl"
  - "-68_ctrl"
  - "-67_ctrl"
  - "-66_ctrl"
  - "-65_ctrl"
  - "-64_ctrl"
  - "-63_ctrl"
  - "-62_ctrl"
  - "-61_ctrl"
  - "-60_ctrl"
  - "-59_ctrl"
  - "-58_ctrl"
  - "-57_ctrl"
  - "-56_ctrl"
  - "-55_ctrl"
  - "-54_ctrl"
  - "-53_ctrl"
  - "-52_ctrl"
  - "-51_ctrl"
  - "-50_ctrl"
  - "-49_ctrl"
  - "-48_ctrl"
  - "-47_ctrl"
  - "-46_ctrl"
  - "-45_ctrl"
  - "-44_ctrl"
  - "-43_ctrl"
  - "-42_ctrl"
  - "-41_ctrl"
  - "-40_ctrl"
  - "-39_ctrl"
  - "-38_ctrl"
  - "-37_ctrl"
  - "-36_ctrl"
  - "-35_ctrl"
  - "-34_ctrl"
  - "-33_ctrl"
  - "-32_ctrl"
  - "-31_ctrl"
  - "-30_ctrl"
  - "-29_ctrl"
  - "-28_ctrl"
  - "-27_ctrl"
  - "-26_ctrl"
  - "-25_ctrl"
  - "-24_ctrl"
  - "-23_ctrl"
  - "-22_ctrl"
  - "-21_ctrl"
  - "-20_ctrl"
  - "-19_ctrl"
  - "-18_ctrl"
  - "-17_ctrl"
  - "-16_ctrl"
  - "-15_ctrl"
  - "-14_ctrl"
  - "-13_ctrl"
  - "-12_ctrl"
  - "-11_ctrl"
  - "-10_ctrl"
  - "-9_ctrl"
  - "-8_ctrl"
  - "-7_ctrl"
  - "-6_ctrl"
  - "-5_ctrl"
  - "-4_ctrl"
  - "-3_ctrl"
  - "-2_ctrl"
  - "-1_ctrl"
  - "0_ctrl"
  - "1_ctrl"
  - "2_ctrl"
  - "3_ctrl"
  - "4_ctrl"
  - "5_ctrl"
  - "6_ctrl"
  - "7_ctrl"
  - "8_ctrl"
  - "9_ctrl"
  - "10_ctrl"
  - "11_ctrl"
  - "12_ctrl"
  - "13_ctrl"
  - "14_ctrl"
  - "15_ctrl"
  - "16_ctrl"
  - "17_ctrl"
  - "18_ctrl"
  - "19_ctrl"
  - "20_ctrl"
  - "21_ctrl"
  - "22_ctrl"
  - "23_ctrl"
  - "24_ctrl"
  - "25_ctrl"
  - "26_ctrl"
  - "27_ctrl"
  - "28_ctrl"
  - "29_ctrl"
  - "30_ctrl"
  - "31_ctrl"
  - "32_ctrl"
  - "33_ctrl"
  - "34_ctrl"
  - "35_ctrl"
  - "36_ctrl"
  - "37_ctrl"
  - "38_ctrl"
  - "39_ctrl"
  - "40_ctrl"
  - "41_ctrl"
  - "42_ctrl"
  - "43_ctrl"
  - "44_ctrl"
  - "45_ctrl"
  - "46_ctrl"
  - "47_ctrl"
  - "48_ctrl"
  - "49_ctrl"
  - "50_ctrl"
  - "51_ctrl"
  - "52_ctrl"
  - "53_ctrl"
  - "54_ctrl"
  - "55_ctrl"
  - "56_ctrl"
  - "57_ctrl"
  - "58_ctrl"
  - "59_ctrl"
  - "60_ctrl"
  - "61_ctrl"
  - "62_ctrl"
  - "63_ctrl"
  - "64_ctrl"
  - "65_ctrl"
  - "66_ctrl"
  - "67_ctrl"
  - "68_ctrl"
  - "69_ctrl"
  - "70_ctrl"
  - "71_ctrl"
  - "72_ctrl"
  - "73_ctrl"
  - "74_ctrl"
  - "75_ctrl"
  - "76_ctrl"
  - "77_ctrl"
  - "78_ctrl"
  - "79_ctrl"
  - "80_ctrl"
  - "81_ctrl"
  - "82_ctrl"
  - "83_ctrl"
  - "84_ctrl"
  - "85_ctrl"
  - "86_ctrl"
  - "87_ctrl"
  - "88_ctrl"
  - "89_ctrl"
  - "90_ctrl"
  - "91_ctrl"
  - "92_ctrl"
  - "93_ctrl"
  - "94_ctrl"
  - "95_ctrl"
  - "96_ctrl"
  - "97_ctrl"
  - "98_ctrl"
  - "99_ctrl"
  - "100_ctrl"
  - "101_ctrl"
  - "102_ctrl"
  - "103_ctrl"
  - "104_ctrl"
  - "105_ctrl"
  - "106_ctrl"
  - "107_ctrl"
  - "108_ctrl"
  - "109_ctrl"
  - "110_ctrl"
  - "111_ctrl"
  - "112_ctrl"
  - "113_ctrl"
  - "114_ctrl"
  - "115_ctrl"
  - "116_ctrl"
  - "117_ctrl"
  - "118_ctrl"
  - "119_ctrl"
  - "120_ctrl"
  - "121_ctrl"
  - "122_ctrl"
  - "123_ctrl"
  - "124_ctrl"
  - "125_ctrl"
  - "126_ctrl"
  - "127_ctrl"
  - "128_ctrl"
  - "129_ctrl"
  - "130_ctrl"
  - "131_ctrl"
  - "132_ctrl"
  - "133_ctrl"
  - "134_ctrl"
  - "135_ctrl"
  - "136_ctrl"
  - "137_ctrl"
  - "138_ctrl"
  - "139_ctrl"
  - "140_ctrl"
  - "141_ctrl"
  - "142_ctrl"
  - "143_ctrl"
  - "144_ctrl"
  - "145_ctrl"
  - "146_ctrl"
  - "147_ctrl"
  - "148_ctrl"
  - "149_ctrl"
  - "150_ctrl"
  - "151_ctrl"
  - "152_ctrl"
  - "153_ctrl"
  - "154_ctrl"
  - "155_ctrl"
  - "156_ctrl"
  - "157_ctrl"
  - "158_ctrl"
  - "159_ctrl"
  - "160_ctrl"
  - "161_ctrl"
  - "162_ctrl"
  - "163_ctrl"
  - "164_ctrl"
  - "165_ctrl"
  - "166_ctrl"
  - "167_ctrl"
  - "168_ctrl"
  - "169_ctrl"
  - "170_ctrl"
  - "171_ctrl"
  - "172_ctrl"
  - "173_ctrl"
  - "174_ctrl"
  - "175_ctrl"
  - "176_ctrl"
  - "177_ctrl"
  - "178_ctrl"
  - "179_ctrl"
  - "180_ctrl"
  - "181_ctrl"
  - "182_ctrl"
  - "183_ctrl"
  - "184_ctrl"
  - "185_ctrl"
  - "186_ctrl"
  - "187_ctrl"
  - "188_ctrl"
  - "189_ctrl"
  - "190_ctrl"
  - "191_ctrl"
  - "192_ctrl"
  - "193_ctrl"
  - "194_ctrl"
  - "195_ctrl"
  - "196_ctrl"
  - "197_ctrl"
  - "198_ctrl"
  - "199_ctrl"
  - "200_ctrl"
  """
}

predict_perturbations_H3K27ac_yaml_contents = {"fusion.yaml":"""  
model_config:
  fc_do: 0.4
  fc_task_dim: 256
  layers:
  - 2
  rb_do: 0.4
  stochastic_depth_p: 0.5
model_type: "mlp-residual"
""",
"globals.yaml": """

checkpoint_interval: 30300 # 100 epochs #60000
sample_interval: 30300 #60000
n_epochs: 120
batch_size: 64
optimizer: "adamw"
lr: 0.0001 #0.0001
device: "cuda"
compute_attributions: false
""",
"input_cnn.yaml": """
input_info:
  input_source: ./inputs/perturbed_landscape_arrays/test_only_H3K27ac_silenced/  #inserted_enhancer_test #inputs/silenced_arrays/silenced_arrays_H2B_S.D 
  #./data/parsed_data/inputs/arrays_train_100bp_no_H3K27ac/ #arrays_train_100bp_no_H3K27ac_uncoupled/
  input_name: gene_expression
  input_type: array

model_config:
  model_type: cnn
  #pre_normalization: "instancenorm"
  model_init_config:
    num_output_features: 512 # before fc_repr_dim
    layers: [4,4]
    kernel_height: 1
    down_stride_width: 2
    first_stride_expansion_width: 1
    first_kernel_expansion_height: 4 #5
    kernel_width: 10  #10
    dilation_factor_width: 2
    dilation_factor_height: 1
    channel_exp_base: 5  #3 #-1
    first_channel_expansion: 2
    rb_do: .3
    stochastic_depth_p: .1
    attention_inclusion_cutoff: 1 #50 #256
    """,
"outputs_2_cond.yaml": """
output_info:
  output_name: expression_output
  output_source: ./targets/perturbed_targets.csv 
  #./data/parsed_data/targets/target_arrays_perturbational_inserted_enhancer.csv #target_arrays_perturbational_S.D.csv #target_arrays_1kbp_401_bins_2_conditions_decareads_abs.csv
  output_type: tabular

model_config: # <- new
  model_type: linear # <- new

output_type_info:
  con_loss_name: "SmoothL1Loss"
  target_con_columns:
  - "-200_ctrl"
  - "-199_ctrl"
  - "-198_ctrl"
  - "-197_ctrl"
  - "-196_ctrl"
  - "-195_ctrl"
  - "-194_ctrl"
  - "-193_ctrl"
  - "-192_ctrl"
  - "-191_ctrl"
  - "-190_ctrl"
  - "-189_ctrl"
  - "-188_ctrl"
  - "-187_ctrl"
  - "-186_ctrl"
  - "-185_ctrl"
  - "-184_ctrl"
  - "-183_ctrl"
  - "-182_ctrl"
  - "-181_ctrl"
  - "-180_ctrl"
  - "-179_ctrl"
  - "-178_ctrl"
  - "-177_ctrl"
  - "-176_ctrl"
  - "-175_ctrl"
  - "-174_ctrl"
  - "-173_ctrl"
  - "-172_ctrl"
  - "-171_ctrl"
  - "-170_ctrl"
  - "-169_ctrl"
  - "-168_ctrl"
  - "-167_ctrl"
  - "-166_ctrl"
  - "-165_ctrl"
  - "-164_ctrl"
  - "-163_ctrl"
  - "-162_ctrl"
  - "-161_ctrl"
  - "-160_ctrl"
  - "-159_ctrl"
  - "-158_ctrl"
  - "-157_ctrl"
  - "-156_ctrl"
  - "-155_ctrl"
  - "-154_ctrl"
  - "-153_ctrl"
  - "-152_ctrl"
  - "-151_ctrl"
  - "-150_ctrl"
  - "-149_ctrl"
  - "-148_ctrl"
  - "-147_ctrl"
  - "-146_ctrl"
  - "-145_ctrl"
  - "-144_ctrl"
  - "-143_ctrl"
  - "-142_ctrl"
  - "-141_ctrl"
  - "-140_ctrl"
  - "-139_ctrl"
  - "-138_ctrl"
  - "-137_ctrl"
  - "-136_ctrl"
  - "-135_ctrl"
  - "-134_ctrl"
  - "-133_ctrl"
  - "-132_ctrl"
  - "-131_ctrl"
  - "-130_ctrl"
  - "-129_ctrl"
  - "-128_ctrl"
  - "-127_ctrl"
  - "-126_ctrl"
  - "-125_ctrl"
  - "-124_ctrl"
  - "-123_ctrl"
  - "-122_ctrl"
  - "-121_ctrl"
  - "-120_ctrl"
  - "-119_ctrl"
  - "-118_ctrl"
  - "-117_ctrl"
  - "-116_ctrl"
  - "-115_ctrl"
  - "-114_ctrl"
  - "-113_ctrl"
  - "-112_ctrl"
  - "-111_ctrl"
  - "-110_ctrl"
  - "-109_ctrl"
  - "-108_ctrl"
  - "-107_ctrl"
  - "-106_ctrl"
  - "-105_ctrl"
  - "-104_ctrl"
  - "-103_ctrl"
  - "-102_ctrl"
  - "-101_ctrl"
  - "-100_ctrl"
  - "-99_ctrl"
  - "-98_ctrl"
  - "-97_ctrl"
  - "-96_ctrl"
  - "-95_ctrl"
  - "-94_ctrl"
  - "-93_ctrl"
  - "-92_ctrl"
  - "-91_ctrl"
  - "-90_ctrl"
  - "-89_ctrl"
  - "-88_ctrl"
  - "-87_ctrl"
  - "-86_ctrl"
  - "-85_ctrl"
  - "-84_ctrl"
  - "-83_ctrl"
  - "-82_ctrl"
  - "-81_ctrl"
  - "-80_ctrl"
  - "-79_ctrl"
  - "-78_ctrl"
  - "-77_ctrl"
  - "-76_ctrl"
  - "-75_ctrl"
  - "-74_ctrl"
  - "-73_ctrl"
  - "-72_ctrl"
  - "-71_ctrl"
  - "-70_ctrl"
  - "-69_ctrl"
  - "-68_ctrl"
  - "-67_ctrl"
  - "-66_ctrl"
  - "-65_ctrl"
  - "-64_ctrl"
  - "-63_ctrl"
  - "-62_ctrl"
  - "-61_ctrl"
  - "-60_ctrl"
  - "-59_ctrl"
  - "-58_ctrl"
  - "-57_ctrl"
  - "-56_ctrl"
  - "-55_ctrl"
  - "-54_ctrl"
  - "-53_ctrl"
  - "-52_ctrl"
  - "-51_ctrl"
  - "-50_ctrl"
  - "-49_ctrl"
  - "-48_ctrl"
  - "-47_ctrl"
  - "-46_ctrl"
  - "-45_ctrl"
  - "-44_ctrl"
  - "-43_ctrl"
  - "-42_ctrl"
  - "-41_ctrl"
  - "-40_ctrl"
  - "-39_ctrl"
  - "-38_ctrl"
  - "-37_ctrl"
  - "-36_ctrl"
  - "-35_ctrl"
  - "-34_ctrl"
  - "-33_ctrl"
  - "-32_ctrl"
  - "-31_ctrl"
  - "-30_ctrl"
  - "-29_ctrl"
  - "-28_ctrl"
  - "-27_ctrl"
  - "-26_ctrl"
  - "-25_ctrl"
  - "-24_ctrl"
  - "-23_ctrl"
  - "-22_ctrl"
  - "-21_ctrl"
  - "-20_ctrl"
  - "-19_ctrl"
  - "-18_ctrl"
  - "-17_ctrl"
  - "-16_ctrl"
  - "-15_ctrl"
  - "-14_ctrl"
  - "-13_ctrl"
  - "-12_ctrl"
  - "-11_ctrl"
  - "-10_ctrl"
  - "-9_ctrl"
  - "-8_ctrl"
  - "-7_ctrl"
  - "-6_ctrl"
  - "-5_ctrl"
  - "-4_ctrl"
  - "-3_ctrl"
  - "-2_ctrl"
  - "-1_ctrl"
  - "0_ctrl"
  - "1_ctrl"
  - "2_ctrl"
  - "3_ctrl"
  - "4_ctrl"
  - "5_ctrl"
  - "6_ctrl"
  - "7_ctrl"
  - "8_ctrl"
  - "9_ctrl"
  - "10_ctrl"
  - "11_ctrl"
  - "12_ctrl"
  - "13_ctrl"
  - "14_ctrl"
  - "15_ctrl"
  - "16_ctrl"
  - "17_ctrl"
  - "18_ctrl"
  - "19_ctrl"
  - "20_ctrl"
  - "21_ctrl"
  - "22_ctrl"
  - "23_ctrl"
  - "24_ctrl"
  - "25_ctrl"
  - "26_ctrl"
  - "27_ctrl"
  - "28_ctrl"
  - "29_ctrl"
  - "30_ctrl"
  - "31_ctrl"
  - "32_ctrl"
  - "33_ctrl"
  - "34_ctrl"
  - "35_ctrl"
  - "36_ctrl"
  - "37_ctrl"
  - "38_ctrl"
  - "39_ctrl"
  - "40_ctrl"
  - "41_ctrl"
  - "42_ctrl"
  - "43_ctrl"
  - "44_ctrl"
  - "45_ctrl"
  - "46_ctrl"
  - "47_ctrl"
  - "48_ctrl"
  - "49_ctrl"
  - "50_ctrl"
  - "51_ctrl"
  - "52_ctrl"
  - "53_ctrl"
  - "54_ctrl"
  - "55_ctrl"
  - "56_ctrl"
  - "57_ctrl"
  - "58_ctrl"
  - "59_ctrl"
  - "60_ctrl"
  - "61_ctrl"
  - "62_ctrl"
  - "63_ctrl"
  - "64_ctrl"
  - "65_ctrl"
  - "66_ctrl"
  - "67_ctrl"
  - "68_ctrl"
  - "69_ctrl"
  - "70_ctrl"
  - "71_ctrl"
  - "72_ctrl"
  - "73_ctrl"
  - "74_ctrl"
  - "75_ctrl"
  - "76_ctrl"
  - "77_ctrl"
  - "78_ctrl"
  - "79_ctrl"
  - "80_ctrl"
  - "81_ctrl"
  - "82_ctrl"
  - "83_ctrl"
  - "84_ctrl"
  - "85_ctrl"
  - "86_ctrl"
  - "87_ctrl"
  - "88_ctrl"
  - "89_ctrl"
  - "90_ctrl"
  - "91_ctrl"
  - "92_ctrl"
  - "93_ctrl"
  - "94_ctrl"
  - "95_ctrl"
  - "96_ctrl"
  - "97_ctrl"
  - "98_ctrl"
  - "99_ctrl"
  - "100_ctrl"
  - "101_ctrl"
  - "102_ctrl"
  - "103_ctrl"
  - "104_ctrl"
  - "105_ctrl"
  - "106_ctrl"
  - "107_ctrl"
  - "108_ctrl"
  - "109_ctrl"
  - "110_ctrl"
  - "111_ctrl"
  - "112_ctrl"
  - "113_ctrl"
  - "114_ctrl"
  - "115_ctrl"
  - "116_ctrl"
  - "117_ctrl"
  - "118_ctrl"
  - "119_ctrl"
  - "120_ctrl"
  - "121_ctrl"
  - "122_ctrl"
  - "123_ctrl"
  - "124_ctrl"
  - "125_ctrl"
  - "126_ctrl"
  - "127_ctrl"
  - "128_ctrl"
  - "129_ctrl"
  - "130_ctrl"
  - "131_ctrl"
  - "132_ctrl"
  - "133_ctrl"
  - "134_ctrl"
  - "135_ctrl"
  - "136_ctrl"
  - "137_ctrl"
  - "138_ctrl"
  - "139_ctrl"
  - "140_ctrl"
  - "141_ctrl"
  - "142_ctrl"
  - "143_ctrl"
  - "144_ctrl"
  - "145_ctrl"
  - "146_ctrl"
  - "147_ctrl"
  - "148_ctrl"
  - "149_ctrl"
  - "150_ctrl"
  - "151_ctrl"
  - "152_ctrl"
  - "153_ctrl"
  - "154_ctrl"
  - "155_ctrl"
  - "156_ctrl"
  - "157_ctrl"
  - "158_ctrl"
  - "159_ctrl"
  - "160_ctrl"
  - "161_ctrl"
  - "162_ctrl"
  - "163_ctrl"
  - "164_ctrl"
  - "165_ctrl"
  - "166_ctrl"
  - "167_ctrl"
  - "168_ctrl"
  - "169_ctrl"
  - "170_ctrl"
  - "171_ctrl"
  - "172_ctrl"
  - "173_ctrl"
  - "174_ctrl"
  - "175_ctrl"
  - "176_ctrl"
  - "177_ctrl"
  - "178_ctrl"
  - "179_ctrl"
  - "180_ctrl"
  - "181_ctrl"
  - "182_ctrl"
  - "183_ctrl"
  - "184_ctrl"
  - "185_ctrl"
  - "186_ctrl"
  - "187_ctrl"
  - "188_ctrl"
  - "189_ctrl"
  - "190_ctrl"
  - "191_ctrl"
  - "192_ctrl"
  - "193_ctrl"
  - "194_ctrl"
  - "195_ctrl"
  - "196_ctrl"
  - "197_ctrl"
  - "198_ctrl"
  - "199_ctrl"
  - "200_ctrl"
  """ }

# Training configs
for file,content in training_yaml_contents.items():
  with open(config_paths[0] / file, 'w') as f:
      f.write(content)

# Test configs
for file,content in test_yaml_contents.items():
  with open(config_paths[1] / file, 'w') as f:
      f.write(content)

# Perturbation configs
# Test configs
for file,content in predict_perturbations_yaml_contents.items():
  with open(config_paths[2] / file, 'w') as f:
      f.write(content)

# Training microc configs
for file,content in training_microc_yaml_contents.items():
  with open(config_paths[3] / file, 'w') as f:
      f.write(content)

# Training microc rotated configs
for file,content in training_microc_rotated_yaml_contents.items():
  with open(config_paths[4] / file, 'w') as f:
      f.write(content)

# Test microc configs
for file,content in test_microc_yaml_contents.items():
  with open(config_paths[5] / file, 'w') as f:
      f.write(content)

# Test microc rotated configs
for file,content in test_microc_rotated_yaml_contents.items():
  with open(config_paths[6] / file, 'w') as f:
      f.write(content)

for file,content in training_attention_yaml_contents.items():
  with open(config_paths[7] / file, 'w') as f:
      f.write(content)

# Test configs
for file,content in test_attention_yaml_contents.items():
  with open(config_paths[8] / file, 'w') as f:
      f.write(content)

# Training configs 2 branch latent 
for file,content in training_microc_latents_yaml_contents.items():
  with open(config_paths[9] / file, 'w') as f:
      f.write(content)

for file,content in predict_perturbations_H3K27ac_yaml_contents.items():
  with open(config_paths[10] / file, 'w') as f:
      f.write(content)



### Create validation split

All samples from chromosome 17 were used as a validation set. Samples from chromosome 4 were in turn used as a final test set.

In [ ]:
def create_validation_ids(path: Path, val_chrom: str = "chr17"):
    """ All genes encoded in chr17 will be used as a validation set."""
    gene_annotations_df = pd.read_csv(path / "Final_gene_annotations.tsv", sep="\t").set_index('ID')
    missing_samples_report = pd.read_csv(path / "missing_samples_report.csv")["Sample"].values
    indices = gene_annotations_df[gene_annotations_df["chr"] == val_chrom].index

    # Prepare the strings to write in the file
    lines = [f"{index}_forward\n{index}_rev\n" for index in indices if index+'_forward' not in missing_samples_report]

    # Write to the text file
    with open(path / f"manual_validation_ids_{val_chrom}.txt", 'w') as file:
        file.writelines(lines)

path = Path("../annotations/")
create_validation_ids(path)

These two ids were removed from the manual validation ids list

ENSMUSG00000055660.9_forward
ENSMUSG00000055660.9_rev


## Training CLASTER
The model can be trained by running the following command:

```bash
eirtrain \
--global_configs ./configurations/conf_pure_conv/globals.yaml \
--input_configs ./configurations/conf_pure_conv/input_cnn.yaml \
--fusion_configs ./configurations/conf_pure_conv/fusion.yaml \
--output_configs ./configurations/conf_pure_conv/outputs_2_cond.yaml
```

We trained CLASTER on an A100 GPU with 80GBs of RAM in a slurm-based high performance cluster (HPC).

This will yield a folder in runs/ with the name of the run and:
- The summary of the configs
- Results:
    - predictions
    - attributions if true
    - latents 
- The saved model
- Serialisations
- Tensorboard logs
- The logging history
- A model summary
- Train and validation logs
...

All of these will be handeled in notebook 3.


Training the 2 branch model with the squared matrices:

```bash
eirtrain \
--global_configs ./configurations/conf_microc_pure_conv/globals.yaml \
--input_configs ./configurations/conf_microc_pure_conv/input_cnn.yaml ./configurations/conf_microc_pure_conv/input_cnn_microc.yaml \
--fusion_configs ./configurations/conf_microc_pure_conv/fusion.yaml \
--output_configs ./configurations/conf_microc_pure_conv/outputs_2_cond.yaml
```

Training the 2 branch model with rotated and cropped Micro-C matrices:

```bash
eirtrain \
--global_configs ./configurations/conf_microc_rotated_pure_conv/globals.yaml \
--input_configs ./configurations/conf_microc_rotated_pure_conv/input_cnn.yaml ./configurations/conf_microc_rotated_pure_conv/input_cnn_microc_rotated.yaml \
--fusion_configs ./configurations/conf_microc_rotated_pure_conv/fusion.yaml \
--output_configs ./configurations/conf_microc_rotated_pure_conv/outputs_2_cond.yaml

Training 1 branch model with attention before MLP:

eirtrain \
--global_configs ./configurations/conf_only_chrom_attention/globals.yaml \
--input_configs ./configurations/conf_only_chrom_attention/input_cnn.yaml \
--fusion_configs ./configurations/conf_only_chrom_attention/fusion.yaml \
--output_configs ./configurations/conf_only_chrom_attention/outputs_2_cond.yaml

Training 2 branch model on test data to get latent representations of Micro-C branch.

eirtrain \
--global_configs ./configurations/conf_microc_pure_conv_latents/globals.yaml \
--input_configs ./configurations/conf_microc_pure_conv_latents/input_cnn.yaml ./configurations/conf_microc_pure_conv_latents/input_cnn_microc.yaml \
--fusion_configs ./configurations/conf_microc_pure_conv_latents/fusion.yaml \
--output_configs ./configurations/conf_microc_pure_conv_latents/outputs_2_cond.yaml

```


## Testing CLASTER

> _Note:_ 
>
>1) You should first create the output folder.
>2) You should modify the name of the saved model according to the split in which you saved it (batch number 60600 for us, ca. 100 epochs) and performance average (0.8161).


In [ ]:
! mkdir -p ../runs/test_runs/gene_expression_only_chrom_pure_conv
! mkdir -p ../runs/test_runs/gene_expression_microc_pure_conv
! mkdir -p ../runs/test_runs/gene_expression_microc_rotated_pure_conv

Test predictions can be obtained by typing the following command.

> _Note:_
For GPU trained models, we need to test using the same device given the serializations.

To run on a SLURM based cluster:

srun --partition=gpuqueue --gres=gpu:a100:1 -- 

```bash
eirpredict \
--global_configs ./configurations/conf_pure_conv_predict/globals.yaml \
--input_configs ./configurations/conf_pure_conv_predict/input_cnn.yaml \
--fusion_configs ./configurations/conf_pure_conv_predict/fusion.yaml \
--output_configs ./configurations/conf_pure_conv_predict/outputs_2_cond.yaml \
--evaluate \
--model_path ./runs/gene_expression_only_chrom_pure_conv/saved_models/gene_expression_only_chrom_pure_conv_model_60600_perf-average=0.8161.pt \
--output_folder ./runs/test_runs/gene_expression_only_chrom_pure_conv 
```

Testing with squared matrices:(we'll plot the inner filter outputs)

```bash
eirpredict \
--global_configs ./configurations/conf_microc_pure_conv_predict/globals.yaml \
--input_configs ./configurations/conf_microc_pure_conv_predict/input_cnn.yaml ./configurations/conf_microc_pure_conv_predict/input_cnn_microc.yaml \
--fusion_configs ./configurations/conf_microc_pure_conv_predict/fusion.yaml \
--output_configs ./configurations/conf_microc_pure_conv_predict/outputs_2_cond.yaml \
--evaluate \
--model_path ./runs/gene_expression_microc_pure_conv/saved_models/gene_expression_microc_pure_conv_model_60600_perf-average=0.8045.pt \
--output_folder ./runs/test_runs/gene_expression_microc_pure_conv 
```

Testing with rotated and cropped matrices:

```bash
eirpredict \
--global_configs ./configurations/conf_microc_rotated_pure_conv_predict/globals.yaml \
--input_configs ./configurations/conf_microc_rotated_pure_conv_predict/input_cnn.yaml ./configurations/conf_microc_rotated_pure_conv_predict/input_cnn_microc_rotated.yaml \
--fusion_configs ./configurations/conf_microc_rotated_pure_conv_predict/fusion.yaml \ 
--output_configs ./configurations/conf_microc_rotated_pure_conv_predict/outputs_2_cond.yaml \ 
--evaluate \ 
--model_path ./runs/gene_expression_microc_rotated_pure_conv/saved_models/ gene_expression_microc_rotated_pure_conv_model_60600_perf-average=0.8196.pt \
--output_folder ./runs/test_runs/gene_expression_microc_rotated_pure_conv 
```

Testing chrom branch with attention:

```bash
eirpredict \
--global_configs ./configurations/conf_only_chrom_attention_predict/globals.yaml \
--input_configs ./configurations/conf_only_chrom_attention_predict/input_cnn.yaml \
--fusion_configs ./configurations/conf_only_chrom_attention_predict/fusion.yaml \
--output_configs ./configurations/conf_only_chrom_attention_predict/outputs_2_cond.yaml \
--evaluate \
--model_path ./runs/gene_expression_only_chrom_attention/saved_models/gene_expression_only_chrom_attention_60600_perf-average=0.8217.pt \
--output_folder ./runs/test_runs/gene_expression_only_chrom_attention 
```

## Predicting _in silico_ perturbed chromatin landscapes

>Note: Make sure to create the output folder beforehand: ./runs/perturbation_runs/gene_expression_only_chrom_pure_conv/

```bash
eirpredict \
--global_configs ./configurations/conf_pure_conv_predict_perturbations/globals.yaml 
--input_configs ./configurations/conf_pure_conv_predict_perturbations/input_cnn.yaml 
--fusion_configs ./configurations/conf_pure_conv_predict_perturbations/fusion.yaml 
--output_configs ./configurations/conf_pure_conv_predict_perturbations/outputs_2_cond.yaml 
--evaluate 
--model_path ./runs/gene_expression_only_chrom_pure_conv/saved_models/gene_expression_only_chrom_pure_conv_model_60600_perf-average=0.8161.pt 
--output_folder ./runs/perturbation_runs/gene_expression_only_chrom_pure_conv \
```


Predicting only H3K27ac modified profiles

```bash
eirpredict \
--global_configs ./configurations/conf_pure_conv_predict_perturbations_H3K27ac/globals.yaml \
--input_configs ./configurations/conf_pure_conv_predict_perturbations_H3K27ac/input_cnn.yaml \
--fusion_configs ./configurations/conf_pure_conv_predict_perturbations_H3K27ac/fusion.yaml \
--output_configs ./configurations/conf_pure_conv_predict_perturbations_H3K27ac/outputs_2_cond.yaml \
--evaluate \
--model_path ./runs/gene_expression_only_chrom_pure_conv/saved_models/gene_expression_only_chrom_pure_conv_model_60600_perf-average=0.8161.pt \
--output_folder ./runs/perturbation_runs/gene_expression_pure_conv_perturbed_only_H3K27ac \
```

That's it! Now we will train a couple of DNA-sequence models on the same samples (mapping DNA to EU-seq now) and compare the results!